# Imports

In [ ]:
from DKI_funcs import *

%load_ext autoreload
%autoreload 2

from statsmodels.stats.multitest import multipletests
from matplotlib.colors import ListedColormap

In [ ]:
Save = False

TrainingSamples = 10_000
InferSamples    = 500
TestSamples = 50

In [ ]:
HCP_dir = '../../HCP_data/'
MS_dir = '../../HCP_data/'

## Load DKI

In [ ]:
i = 1
fdwi = HCP_dir+'Pat'+str(i)+'/diff_1k.nii.gz'
bvalloc = HCP_dir+'Pat'+str(i)+'/bvals_1k.txt'
bvecloc = HCP_dir+'Pat'+str(i)+'/bvecs_1k.txt'

fdwi3 = HCP_dir+'Pat'+str(i)+'/diff_3k.nii.gz'
bvalloc3 = HCP_dir+'Pat'+str(i)+'/bvals_3k.txt'
bvecloc3 = HCP_dir+'Pat'+str(i)+'/bvecs_3k.txt'

bvalsHCP = np.loadtxt(bvalloc)
bvecsHCP = np.loadtxt(bvecloc)
gtabHCP = gradient_table(bvals = bvalsHCP, bvecs = bvecsHCP)

bvalsHCP3 = np.loadtxt(bvalloc3)
bvecsHCP3 = np.loadtxt(bvecloc3)
gtabHCP3 = gradient_table(bvals = bvalsHCP3, bvecs = bvecsHCP3)

gtabExt  = gradient_table(bvals=np.hstack((bvalsHCP,bvalsHCP3)), bvecs=np.vstack((bvecsHCP,bvecsHCP3)))
bvals2 = np.copy(gtabExt.bvals)
bvals2[bvals2==3000] = 4000
bvals2[bvals2==3000] = 4000
gtabExt2  = gradient_table(bvals2, gtabExt.bvecs)

data, affine, img = load_nifti(fdwi, return_img=True)
data, affine = reslice(data, affine, (1.5,1.5,1.5), (2.5,2.5,2.5))
axial_middle = data.shape[2] // 2
maskdata, mask = median_otsu(data, vol_idx=range(10, 50), median_radius=3,
                             numpass=1, autocrop=False, dilate=2)

data3, affine, img = load_nifti(fdwi3, return_img=True)
data3, affine = reslice(data3, affine, (1.5,1.5,1.5), (2.5,2.5,2.5))
# Get the indices of True values
true_indices = np.argwhere(mask)

# Determine the minimum and maximum indices along each dimension
min_coords = true_indices.min(axis=0)
max_coords = true_indices.max(axis=0)

maskdata  = maskdata[min_coords[0]:max_coords[0]+1,min_coords[1]:max_coords[1]+1,min_coords[2]:max_coords[2]+1]
maskdata3 = data3[min_coords[0]:max_coords[0]+1,min_coords[1]:max_coords[1]+1,min_coords[2]:max_coords[2]+1]

TestData = np.concatenate([maskdata[:, :, axial_middle, :],maskdata3[:, :, axial_middle, :]],axis=-1)
FlatTD = TestData.reshape(maskdata.shape[0]*maskdata.shape[1],138)
FlatTD = FlatTD[FlatTD[:,:69].sum(axis=-1)>0]
FlatTD = FlatTD[~np.array(FlatTD<0).any(axis=-1)]

dkimodel = dki.DiffusionKurtosisModel(gtabExt)
tenfit = dkimodel.fit(FlatTD)
DKIHCP = tenfit.kt
DTIHCP = tenfit.lower_triangular()
DKIFull = np.array(DKIHCP)
DTIFull = np.array(DTIHCP)

DTIFilt1 = DTIFull[(abs(DKIFull)<10).all(axis=1)]
DKIFilt1 = DKIFull[(abs(DKIFull)<10).all(axis=1)]
DTIFilt = DTIFilt1[(DKIFilt1>-3/7).all(axis=1)]
DKIFilt = DKIFilt1[(DKIFilt1>-3/7).all(axis=1)]

TrueMets = []
FA       = []
for (dt,kt) in tqdm(zip(DTIFilt,DKIFilt)):
    TrueMets.append(DKIMetrics(dt,kt))
    FA.append(FracAni(np.linalg.eigh(vals_to_mat(dt))[0],np.mean(np.linalg.eigh(vals_to_mat(dt))[0])))
TrueMets = np.array(TrueMets)
TrueFA = np.array(FA)

In [ ]:
# Full fit
DT1_full,DT2_full = FitDT(DTIFilt,1)
x4_full,R1_full,x2_full,R2_full = FitKT(DKIFilt,1)

# LowFA Fit
DT1_lfa,DT2_lfa = FitDT(DTIFilt[TrueMets[:,-1]<0.3,:],1)
x4_lfa,R1_lfa,x2_lfa,R2_lfa = FitKT(DKIFilt[TrueMets[:,-1]<0.3,:],1)

# HighFA Fit
DT1_hfa,DT2_hfa = FitDT(DTIFilt[TrueMets[:,-1]>0.7,:],1)
x4_hfa,R1_hfa,x2_hfa,R2_hfa = FitKT(DKIFilt[TrueMets[:,-1]>0.7,:],1)

# UltraLowFA Fit
DT1_ulfa,DT2_ulfa = FitDT(DTIFilt[TrueMets[:,-1]<0.1,:],1)
x4_ulfa,R1_ulfa,x2_ulfa,R2_ulfa = FitKT(DKIFilt[TrueMets[:,-1]<0.1,:],1)

# HigherAK Fit
DT1_hak,DT2_hak = FitDT(DTIFilt[TrueMets[:,1]>0.9,:],1)
x4_hak,R1_hak,x2_hak,R2_hak = FitKT(DKIFilt[TrueMets[:,1]>0.9,:],1)

## Network

In [ ]:
if os.path.exists(NetworkDir+"DKI_Network.pickle"):
    with open(NetworkDir+"DKI_Network.pickle", "rb") as handle:
        Network = pickle.load(handle)
        print('loaded')
else:
    torch.manual_seed(1)
    np.random.seed(1)
    DT = []
    KT = []
    S0 = []
    DT2,KT2 = GenDTKT([DT1_lfa,DT2_lfa],[x4_lfa,R1_lfa,x2_lfa,R2_lfa],12,int(1300*400))
    DT3,KT3 = GenDTKT([DT1_hfa,DT2_hfa],[x4_hfa,R1_hfa,x2_hfa,R2_hfa],12,int(1300*400))
    DT5,KT5 = GenDTKT([DT1_hak,DT2_hak],[x4_hak,R1_hak,x2_hak,R2_hak],12,int(2600*400))   

    DT = np.vstack([DT2,DT3,DT5])
    KT = np.vstack([KT2,KT3,KT5])

    S0Dist = BoxUniform(low=torch.tensor([lower_S0]), high=torch.tensor([upper_S0]))

    S0 = S0Dist.sample([len(DT)])
    bval_choice = np.random.choice([0,1],len(DT),replace=True)
    S0 = np.array(S0).reshape(len(S0),1)

    indx = np.arange(len(KT))
    Obs = np.zeros([len(KT),len(gtabExt.bvecs)])
    kk = 0
    while len(indx)>0:
        for i in tqdm(indx,position=0,leave=True): 
            gtab = [gtabExt,gtabExt2][bval_choice[i]]
            Obs[i] = CustomDKISimulator(DT[i],KT[i],gtab,S0[i],50)

        indxNew = []
        for i,O in enumerate(Obs):
            if (O>4*np.array(S0[i])).any() or (O<0).any():
                indxNew.append(i)
        KT[indxNew] = KT[indxNew]/2
        DT[indxNew] = GenDTKT([DT1_full,DT2_full],[x4_full,R1_full,x2_full,R2_full],kk,1)[0]

        indx = indxNew
        kk+=1
    Par = np.hstack([DT,KT,S0])

    GtabChoice = []
    for i in range(int(len(DT)*0.8)):
        R1 = np.random.choice(np.where(gtabExt.bvals==1000)[0],6,replace=False)
        R2 = np.random.choice(np.where(gtabExt.bvals==3000)[0],15,replace=False)
        GtabChoice.append(np.hstack([0,R1,R2]))
    for i in range(int(len(DT)*0.2)):
        R1 = np.random.choice(np.where(gtabExt.bvals==1000)[0],np.random.randint(6,64),replace=False)
        R2 = np.random.choice(np.where(gtabExt.bvals==3000)[0],np.random.randint(21,64),replace=False)
        GtabChoice.append(np.hstack([0,R1,R2]))
    Obs_feats = np.array([DKIFeatures([gtabExt,gtabExt2][b].bvecs[R],[gtabExt,gtabExt2][b].bvals[R],O[R]) for O,R,b in zip(Obs,GtabChoice,bval_choice)])
    Obs_feats = torch.tensor(Obs_feats).float()
    Par = torch.tensor(Par).float()

    inference = SNPE(device='mps')

    # generate simulations and pass to the inference object
    inference = inference.append_simulations(Par, Obs_feats,data_device='cpu')

    # train the density estim ator and build the posterior
    density_estimator = inference.train(training_batch_size = 1024)

    low = Par.min(axis=0)[0] - 10*torch.sign(Par.min(axis=0)[0])*Par.min(axis=0)[0]
    high = Par.max(axis=0)[0] + 10*Par.max(axis=0)[0]

    prior_bounds = BoxUniform(low=low, high=high)
    Network = DirectPosterior(density_estimator.cpu(), prior=prior_bounds)

# Figure 2

In [ ]:
i=3
fdwi = HCP_dir + 'Pat'+str(i)+'/diff_1k.nii.gz'
bvalloc = HCP_dir + 'Pat'+str(i)+'/bvals_1k.txt'
bvecloc = HCP_dir + 'Pat'+str(i)+'/bvecs_1k.txt'

fdwi3 = HCP_dir + 'Pat'+str(i)+'/diff_3k.nii.gz'
bvalloc3 = HCP_dir + 'Pat'+str(i)+'/bvals_3k.txt'
bvecloc3 = HCP_dir + 'Pat'+str(i)+'/bvecs_3k.txt'

bvalsHCP = np.loadtxt(bvalloc)
bvecsHCP = np.loadtxt(bvecloc)
gtabHCP = gradient_table(bvals = bvalsHCP, bvecs = bvecsHCP)

bvalsHCP3 = np.loadtxt(bvalloc3)
bvecsHCP3 = np.loadtxt(bvecloc3)
gtabHCP3 = gradient_table(bvals = bvalsHCP3, bvecs = bvecsHCP3)

gtabExt  = gradient_table(bvals = np.hstack((bvalsHCP,bvalsHCP3)), bvecs = np.vstack((bvecsHCP,bvecsHCP3)))

# Choose the first point (arbitrary starting point, e.g., the first gradient)
selected_indices = [1]
distance_matrix = squareform(pdist(bvecsHCP))
# Iteratively select the point furthest from the current selection
for _ in range(5):  # We need 7 points in total, and one is already selected
    remaining_indices = list(set(range(len(bvecsHCP))) - set(selected_indices))
    
    # Calculate the minimum distance to the selected points for each remaining point
    min_distances = np.min(distance_matrix[remaining_indices][:, selected_indices], axis=1)
    
    # Select the point with the maximum minimum distance
    next_index = remaining_indices[np.argmax(min_distances)]
    selected_indices.append(next_index)

selected_indices7 = [0]+selected_indices

bvalsHCP7_1 = bvalsHCP[selected_indices7]
bvecsHCP7_1 = bvecsHCP[selected_indices7]

# Choose the first point (arbitrary starting point, e.g., the first gradient)
selected_indices = [0]

temp_bvecs = bvecsHCP3[bvalsHCP3>0]
temp_bvals = bvalsHCP3[bvalsHCP3>0]
distance_matrix = squareform(pdist(temp_bvecs))
# Iteratively select the point furthest from the current selection
for _ in range(14):  # We need 7 points in total, and one is already selected
    remaining_indices = list(set(range(len(temp_bvecs))) - set(selected_indices))
    
    # Calculate the minimum distance to the selected points for each remaining point
    min_distances = np.min(distance_matrix[remaining_indices][:, selected_indices], axis=1)
    
    # Select the point with the maximum minimum distance
    next_index = remaining_indices[np.argmax(min_distances)]
    selected_indices.append(next_index)

bvalsHCP7_3 = temp_bvals[selected_indices]
bvecsHCP7_3 = temp_bvecs[selected_indices]

true_indx = []
for b in bvecsHCP7_3:
    true_indx.append(np.linalg.norm(b-bvecsHCP3,axis=1).argmin())
true_indx = selected_indices7+[t+69 for t in true_indx]
gtabHCP7 = gradient_table(bvals = np.hstack((bvalsHCP7_1,bvalsHCP7_3)),bvecs = np.vstack((bvecsHCP7_1,bvecsHCP7_3)))


## e

In [ ]:
torch.manual_seed(2)
np.random.seed(2)
j = 1
vL = torch.tensor([0.2*j])
vS = torch.tensor([0.01*j])  

DT,KT = GenDTKT([DT1_full,DT2_full],[x4_full,R1_full,x2_full,R2_full],2,1)

tObs = CustomDKISimulator(DT.squeeze(),KT.squeeze(),gtabExt,200,20)
tTrue = CustomDKISimulator(DT.squeeze(),KT.squeeze(),gtabExt,200,None)

In [ ]:
torch.manual_seed(1)
np.random.seed(1)
posterior_samples_1 = Network.sample((InferSamples,), x=DKIFeatures(gtabExt.bvecs,gtabExt.bvals,tObs),show_progress_bars=True)
GuessDKI = posterior_samples_1.mean(axis=0)
GuessSig = CustomDKISimulator(GuessDKI[:6],GuessDKI[6:],gtabExt,200)
plt.subplots(figsize=(6,1))
plt.plot(tTrue,lw=2,c='k',label='True signal')
plt.plot(GuessSig,lw=2,c=SBIFit,ls='--',label='SBI Recon.')
plt.axis('off')
legend = plt.legend(ncols=2,loc=1,bbox_to_anchor =  (1,1.95),fontsize=26,columnspacing=0.3,handlelength=0.4,handletextpad=0.1)
for handle in legend.get_lines():
    handle.set_linewidth(6)  # Set desired linewidth

In [ ]:
dkimodel = dki.DiffusionKurtosisModel(gtabExt,fit_method='NLLS')
tenfit = dkimodel.fit(tObs)
plt.subplots(figsize=(6,1))
plt.plot(tTrue,lw=2,c='k')
plt.plot(tenfit.predict(gtabExt,200),lw=2,c=WLSFit,ls='--',label='NLLS Recon.')
plt.axis('off')
legend = plt.legend(ncols=2,loc=1,bbox_to_anchor =  (0.9,1.95),fontsize=26,columnspacing=0.3,handlelength=0.4,handletextpad=0.1)
for handle in legend.get_lines():
    handle.set_linewidth(6)  # Set desired linewidth

## f

In [ ]:
torch.manual_seed(1)
np.random.seed(1)
DT2,KT2 = GenDTKT([DT1_lfa,DT2_lfa],[x4_lfa,R1_lfa,x2_lfa,R2_lfa],1,50)
DT3,KT3 = GenDTKT([DT1_hfa,DT2_hfa],[x4_hfa,R1_hfa,x2_hfa,R2_hfa],1,50)
DT5,KT5 = GenDTKT([DT1_hak,DT2_hak],[x4_hak,R1_hak,x2_hak,R2_hak],12,100)

SampsDT = np.vstack([DT2,DT3,DT5])
SampsKT = np.vstack([KT2,KT3,KT5])

Samples  = []
for Sd,Sk in zip(SampsDT,SampsKT):
    Samples.append([CustomDKISimulator(Sd,Sk,gtabExt, S0=200,snr=scale) for scale in [20,10,5,2]])

Samples = np.array(Samples)

In [ ]:
torch.manual_seed(10)
ErrorFull = []
for k in tqdm(range(4)):
    ErrorN2 = []
    ENoise = []
    for i in range(200):
        posterior_samples_1 = Network.sample((InferSamples,), x=DKIFeatures(gtabExt.bvecs,gtabExt.bvals,Samples[i,k,:]),show_progress_bars=False)
        GuessSBI = posterior_samples_1.mean(axis=0)
        
        ErrorN2.append(DKIErrors(GuessSBI[:6],GuessSBI[6:],SampsDT[i],SampsKT[i]))
    ErrorFull.append(ErrorN2)

Error_s = []
dkimodel = dki.DiffusionKurtosisModel(gtabExt,fit_method='NLLS')

for k in tqdm(range(4)):
    ErrorN2 = []
    ENoise = []
    for i in range(200):
        tenfit = dkimodel.fit(Samples[i,k,:])
        
        ErrorN2.append(DKIErrors(tenfit.lower_triangular(),tenfit.kt,SampsDT[i],SampsKT[i]))
    Error_s.append(ErrorN2)



In [ ]:
ErrorFull = np.array(ErrorFull)
Error_s = np.array(Error_s)
ErrorNames = ['MK Error', 'AK Error', 'RK Error', 'MKT Error', 'KFA Error']
fig,ax = plt.subplots(1,3,figsize=(13.5,3))
for i in range(3):
    plt.sca(ax[i])
    g_pos = np.array([1.3,2.3,3.3,4.3])
    colors = ['lightseagreen','lightseagreen','lightseagreen','lightseagreen']
    colors2 = ['paleturquoise','paleturquoise','paleturquoise','paleturquoise']
    BoxPlots(ErrorFull[:,:,i],g_pos,colors,colors2,ax[i],widths=0.3,scatter=False)
    g_pos = np.array([1,2,3,4])
    colors = ['sandybrown','sandybrown','sandybrown','sandybrown']
    colors2 = ['peachpuff','peachpuff','peachpuff','peachpuff']
    BoxPlots(Error_s[:,:,i],g_pos,colors,colors2,ax[i],widths=0.3,scatter=False)
    plt.xticks([1.15, 2.15, 3.15, 4.15,],[20,10,5,2],fontsize=32)
    plt.gca().ticklabel_format(axis='y',style='sci',scilimits=(-1,1))
    plt.grid(axis='y')
    plt.yticks(fontsize=32)
    
    if(i==0):
        handles = [
            Line2D([0], [0], color=SBIFit, lw=4, label='SBI'),Line2D([0], [0], color=WLSFit, lw=4, label='NLLS'),  # Adjust color as per the actual plot color
        ]
        # Add the legenda
        plt.legend(handles=handles,loc=2, bbox_to_anchor=(-0.1,1.1),
                   fontsize=36,columnspacing=0.3,handlelength=0.6,handletextpad=0.3)

## g

In [ ]:
torch.manual_seed(1)
np.random.seed(1)
posterior_samples_1 = Network.sample((InferSamples,), x=DKIFeatures(gtabHCP7.bvecs,gtabHCP7.bvals,tObs[true_indx]),show_progress_bars=True)
GuessDKI = posterior_samples_1.mean(axis=0)
GuessSig = CustomDKISimulator(GuessDKI[:6],GuessDKI[6:],gtabExt,200)
plt.subplots(figsize=(6,1))
plt.plot(tTrue,lw=2,c='k',label='True signal')
plt.plot(GuessSig,lw=2,c=SBIFit,ls='--',label='SBI Recon.')
plt.axis('off')
plt.fill_betweenx(np.arange(0,500,50),0*np.ones(10),7*np.ones(10),color='gray',alpha=0.5)
plt.fill_betweenx(np.arange(0,500,50),64*np.ones(10),79*np.ones(10),color='gray',alpha=0.5)
plt.ylim(-9.996985449425491, 209.99985644997255)
legend = plt.legend(ncols=2,loc=1,bbox_to_anchor =  (1,1.95),fontsize=26,columnspacing=0.3,handlelength=0.4,handletextpad=0.1)
for handle in legend.get_lines():
    handle.set_linewidth(6)  # Set desired linewidth

In [ ]:
dkimodel = dki.DiffusionKurtosisModel(gtabHCP7,fit_method='NLLS')
tenfit = dkimodel.fit(tObs[true_indx])
plt.subplots(figsize=(6,1))
plt.plot(tTrue,lw=2,c='k')
plt.plot(tenfit.predict(gtabExt,200),lw=2,c=WLSFit,ls='--',label='NLLS Recon.')
plt.axis('off')
legend = plt.legend(ncols=2,loc=1,bbox_to_anchor =  (0.9,1.95),fontsize=26,columnspacing=0.3,handlelength=0.4,handletextpad=0.1)
for handle in legend.get_lines():
    handle.set_linewidth(6)  # Set desired linewidth
plt.fill_betweenx(np.arange(0,500,50),0*np.ones(10),7*np.ones(10),color='gray',alpha=0.5)
plt.fill_betweenx(np.arange(0,500,50),64*np.ones(10),79*np.ones(10),color='gray',alpha=0.5)
plt.ylim(-9.996985449425491, 209.99985644997255)

## h

In [ ]:
Samples7  = []
for Sd,Sk in zip(SampsDT,SampsKT):
    Samples7.append([CustomDKISimulator(Sd,Sk,gtabHCP7, S0=200,snr=scale) for scale in [20,10,5,2]])

Samples7 = np.array(Samples7)

In [ ]:
torch.manual_seed(10)
ErrorMin = []
for k in tqdm(range(4)):
    ErrorN2 = []
    ENoise = []
    for i in range(200):
        posterior_samples_1 = Network.sample((InferSamples,), x=DKIFeatures(gtabHCP7.bvecs,gtabHCP7.bvals,Samples7[i,k,:]),show_progress_bars=False)
        GuessSBI = posterior_samples_1.mean(axis=0)
        
        ErrorN2.append(DKIErrors(GuessSBI[:6],GuessSBI[6:],SampsDT[i],SampsKT[i]))
    ErrorMin.append(ErrorN2)

Error_s_min = []
dkimodel = dki.DiffusionKurtosisModel(gtabHCP7,fit_method='NLLS')

for k in tqdm(range(4)):
    ErrorN2 = []
    ENoise = []
    for i in range(200):
        tenfit = dkimodel.fit(Samples7[i,k,:])
        
        ErrorN2.append(DKIErrors(tenfit.lower_triangular(),tenfit.kt,SampsDT[i],SampsKT[i]))
    Error_s_min.append(ErrorN2)



In [ ]:
ErrorFull = np.array(ErrorMin)
Error_s = np.array(Error_s_min)
ErrorNames = ['MK Error', 'AK Error', 'RK Error', 'MKT Error', 'KFA Error']
fig,ax = plt.subplots(1,3,figsize=(13.5,3))
for i in range(3):
    plt.sca(ax[i])
    g_pos = np.array([1.3,2.3,3.3,4.3])
    colors = ['lightseagreen','lightseagreen','lightseagreen','lightseagreen']
    colors2 = ['paleturquoise','paleturquoise','paleturquoise','paleturquoise']
    BoxPlots(ErrorFull[:,:,i],g_pos,colors,colors2,ax[i],widths=0.3,scatter=False)
    g_pos = np.array([1,2,3,4])
    colors = ['sandybrown','sandybrown','sandybrown','sandybrown']
    colors2 = ['peachpuff','peachpuff','peachpuff','peachpuff']
    BoxPlots(Error_s[:,:,i],g_pos,colors,colors2,ax[i],widths=0.3,scatter=False)
    plt.xticks([1.15, 2.15, 3.15, 4.15,],[20,10,5,2],fontsize=32)
    plt.gca().ticklabel_format(axis='y',style='sci',scilimits=(-1,1))
    plt.grid(axis='y')
    plt.yticks(fontsize=32)

# Figure 4

## a

In [ ]:
i=3
fdwi = HCP_dir+'/Pat'+str(i)+'/diff_1k.nii.gz'

data, affine, img = load_nifti(fdwi, return_img=True)
data, affine = reslice(data, affine, (1.5,1.5,1.5), (2.5,2.5,2.5))
maskdata, mask = median_otsu(data, vol_idx=range(10, 50), median_radius=4,
                             numpass=1, autocrop=False, dilate=2)
_, mask2 = median_otsu(data, vol_idx=range(10, 50), median_radius=4,
                             numpass=1, autocrop=True, dilate=2)


data3, affine, img = load_nifti(fdwi3, return_img=True)
data3, affine = reslice(data3, affine, (1.5,1.5,1.5), (2.5,2.5,2.5))
# Get the indices of True values
true_indices = np.argwhere(mask)

# Determine the minimum and maximum indices along each dimension
min_coords = true_indices.min(axis=0)
max_coords = true_indices.max(axis=0)

maskdata  = maskdata[min_coords[0]:max_coords[0]+1,min_coords[1]:max_coords[1]+1,min_coords[2]:max_coords[2]+1]
axial_middle = maskdata.shape[2] // 2
maskdata3 = data3[min_coords[0]:max_coords[0]+1,min_coords[1]:max_coords[1]+1,min_coords[2]:max_coords[2]+1]

TestData = np.concatenate([maskdata[:, :, axial_middle, :],maskdata3[:, :, axial_middle, :]],axis=-1)
TestData4D = np.concatenate([maskdata,maskdata3],axis=-1)

floor = np.clip(TestData4D.min(axis=-1),-np.inf,0)
TestData4D_2 = np.copy(TestData4D)
TestData4D_2[floor <0 ] = TestData4D[floor < 0] + abs(floor)[floor <0 ,None] + 1e-5

cutout = np.sum(TestData4D[:,:,axial_middle,:69], axis=-1) != 0

In [ ]:
# Compute the mask where the sum is not zero
mask = np.sum(TestData4D_2[:,:,axial_middle,:69], axis=-1) != 0

# Get the indices where mask is True
indices = np.argwhere(mask)

def optimize_chunk(pixels):
    results = []
    for i, j in pixels:
        samples = Network.sample((500,), x=DKIFeatures(gtabExt.bvecs,gtabExt.bvals,TestData4D_2[i,j,axial_middle, :]),show_progress_bars=False)
        results.append((i, j, samples.mean(axis=0)))
    return results

chunked_indices = [indices[i:i+ChunkSize] for i in range(0, len(indices), ChunkSize)]
results = Parallel(n_jobs=8)(
    delayed(optimize_chunk)(chunk) for chunk in tqdm(chunked_indices,position=0,leave=True)
)


# Initialize NoiseEst with the appropriate shape
ArrShape = mask.shape

NoiseEst = np.zeros([62, 68 ,22])
for chunk in tqdm(results,position=0,leave=True):
    for i, j, x in chunk:
        NoiseEst[i, j] = x
NoiseEst2 =  np.zeros_like(NoiseEst)
for i in range(62):
    for j in range(68):    
        NoiseEst2[i,j] = np.hstack([mat_to_vals(clip_negative_eigenvalues(vals_to_mat(NoiseEst[i,j]))),NoiseEst[i,j,6:]])

MK_SBIFull  = np.zeros([62, 68])
AK_SBIFull  = np.zeros([62, 68])
RK_SBIFull  = np.zeros([62, 68])
MKT_SBIFull = np.zeros([62, 68])
KFA_SBIFull = np.zeros([62, 68])
for i in tqdm(range(62),position=0,leave=True):
    for j in range(68):
        Metrics = DKIMetrics(NoiseEst2[i,j][:6],NoiseEst2[i,j][6:21])
        MK_SBIFull[i,j] = Metrics[0]
        AK_SBIFull[i,j] = Metrics[1]
        RK_SBIFull[i,j] = Metrics[2]
        MKT_SBIFull[i,j] = Metrics[3]
        KFA_SBIFull[i,j] = Metrics[4]
KFA_SBIFull[np.isnan(KFA_SBIFull)] = 1

In [ ]:
tnorm = TwoSlopeNorm(vmin=-0,vcenter = 0.6,vmax=1.4)
temp = np.copy(MK_SBIFull)
temp[~cutout] = math.nan
plt.imshow(temp.T,norm=tnorm,cmap='hot')
plt.axis('off')
plt.show()

temp = np.copy(AK_SBIFull)
temp[~cutout] = math.nan
plt.imshow(temp.T,norm=tnorm,cmap='hot')
plt.axis('off')
plt.show()

temp = np.copy(RK_SBIFull)
temp[~cutout] = math.nan
plt.imshow(temp.T,norm=tnorm,cmap='hot')
plt.axis('off')
cbar = plt.colorbar(fraction=0.032, pad=0.04)
cbar.ax.set_ylim(0,1.4)
plt.show()

In [ ]:
# Compute the mask where the sum is not zero
mask = np.sum(TestData4D_2[:,:,axial_middle,:69], axis=-1) != 0

# Get the indices where mask is True
indices = np.argwhere(mask)

def optimize_chunk(pixels):
    results = []
    for i, j in pixels:
        samples = Network.sample((500,), x=DKIFeatures(gtabHCP7.bvecs,gtabHCP7.bvals,TestData4D_2[i,j,axial_middle, true_indx]),show_progress_bars=False)
        results.append((i, j, samples.mean(axis=0)))
    return results

chunked_indices = [indices[i:i+ChunkSize] for i in range(0, len(indices), ChunkSize)]
results = Parallel(n_jobs=8)(
    delayed(optimize_chunk)(chunk) for chunk in tqdm(chunked_indices,position=0,leave=True)
)


# Initialize NoiseEst with the appropriate shape
ArrShape = mask.shape

NoiseEst = np.zeros([62, 68 ,22])
for chunk in tqdm(results,position=0,leave=True):
    for i, j, x in chunk:
        NoiseEst[i, j] = x
NoiseEst2 =  np.zeros_like(NoiseEst)
for i in range(62):
    for j in range(68):    
        NoiseEst2[i,j] = np.hstack([mat_to_vals(clip_negative_eigenvalues(vals_to_mat(NoiseEst[i,j]))),NoiseEst[i,j,6:]])

MK_SBI7  = np.zeros([62, 68])
AK_SBI7  = np.zeros([62, 68])
RK_SBI7  = np.zeros([62, 68])
MKT_SBI7 = np.zeros([62, 68])
KFA_SBI7 = np.zeros([62, 68])
for i in tqdm(range(62),position=0,leave=True):
    for j in range(68):
        Metrics = DKIMetrics(NoiseEst2[i,j][:6],NoiseEst2[i,j][6:21])
        MK_SBI7[i,j] = Metrics[0]
        AK_SBI7[i,j] = Metrics[1]
        RK_SBI7[i,j] = Metrics[2]
        MKT_SBI7[i,j] = Metrics[3]
        KFA_SBI7[i,j] = Metrics[4]
KFA_SBI7[np.isnan(KFA_SBI7)] = 1

In [ ]:
tnorm = TwoSlopeNorm(vmin=-0,vcenter = 0.6,vmax=1.4)
temp = np.copy(MK_SBI7)
temp = gaussian_filter(temp, sigma=0.5)
temp[~cutout] = math.nan
plt.imshow(temp.T,norm=tnorm,cmap='hot')
plt.axis('off')
plt.show()

temp = np.copy(AK_SBI7)
temp = gaussian_filter(temp, sigma=0.5)
temp[~cutout] = math.nan
plt.imshow(temp.T,norm=tnorm,cmap='hot')
plt.axis('off')
plt.show()

temp = np.copy(RK_SBI7)
temp = gaussian_filter(temp, sigma=0.5)
temp[~cutout] = math.nan
plt.imshow(temp.T,norm=tnorm,cmap='hot')
plt.axis('off')
cbar = plt.colorbar(fraction=0.032, pad=0.04)
cbar.ax.set_ylim(0,1.4)
plt.show()

In [ ]:
ticks = [0,1,2]
data = np.abs((MK_SBIFull-MK_SBI7)*cutout).T
data[~cutout.T] = np.nan
norm = TwoSlopeNorm(vmin=0, vcenter=1, vmax=2)
plt.imshow(data,cmap='Reds',norm=norm)
plt.axis('off')
cbar = plt.colorbar(ticks=ticks)
cbar.formatter.set_powerlimits((0, 0))
plt.show()


data = np.abs((AK_SBIFull-AK_SBI7)*cutout).T
data[~cutout.T] = np.nan
norm = TwoSlopeNorm(vmin=0, vcenter=np.nanmax(data)/2, vmax=np.nanmax(data))
plt.imshow(data,cmap='Reds',norm=norm)
plt.axis('off')
cbar = plt.colorbar(ticks=ticks)
cbar.formatter.set_powerlimits((0, 0))
plt.show()

data = np.abs((RK_SBIFull-RK_SBI7)*cutout).T
data[~cutout.T] = np.nan
norm = TwoSlopeNorm(vmin=0, vcenter=1,vmax=2)
plt.imshow(data,cmap='Reds',norm=norm)
plt.axis('off')
cbar = plt.colorbar()
cbar.formatter.set_powerlimits((0, 0))
plt.show()

## b

In [ ]:
dkimodelNL = dki.DiffusionKurtosisModel(gtabExt,fit_method='NLLS')
dkifitNL = dkimodelNL.fit(TestData[:,:,:])

MK_NLFull  = dkifitNL.mk() #np.zeros([62, 68])
AK_NLFull  = dkifitNL.ak()# np.zeros([62, 68])
RK_NLFull  = dkifitNL.rk()# np.zeros([62, 68])

In [ ]:
tnorm = TwoSlopeNorm(vmin=-0,vcenter = 0.6,vmax=1.4)
temp = np.copy(MK_NLFull)
temp[~cutout] = math.nan
plt.imshow(temp.T,norm=tnorm,cmap='hot')
plt.axis('off')
plt.show()

temp = np.copy(AK_NLFull)
temp[~cutout] = math.nan
plt.imshow(temp.T,norm=tnorm,cmap='hot')
plt.axis('off')
plt.show()

temp = np.copy(RK_NLFull)
temp[~cutout] = math.nan
plt.imshow(temp.T,norm=tnorm,cmap='hot')
plt.axis('off')
cbar = plt.colorbar(fraction=0.032, pad=0.04)
cbar.ax.set_ylim(0,1.4)
plt.show()

In [ ]:
dkimodelNL = dki.DiffusionKurtosisModel(gtabHCP7,fit_method='NLLS')
dkifitNL = dkimodelNL.fit(TestData[:,:,true_indx])
MK_NL7  = dkifitNL.mk() #np.zeros([62, 68])
AK_NL7  = dkifitNL.ak()# np.zeros([62, 68])
RK_NL7  = dkifitNL.rk()# np.zeros([62, 68])

In [ ]:
tnorm = TwoSlopeNorm(vmin=-0,vcenter = 0.6,vmax=1.4)
temp = np.copy(MK_NL7)
temp[~cutout] = math.nan
plt.imshow(temp.T,norm=tnorm,cmap='hot')
plt.axis('off')
plt.show()

temp = np.copy(AK_NL7)
temp[~cutout] = math.nan
plt.imshow(temp.T,norm=tnorm,cmap='hot')
plt.axis('off')
plt.show()

temp = np.copy(RK_NL7)
temp[~cutout] = math.nan
plt.imshow(temp.T,norm=tnorm,cmap='hot')
plt.axis('off')
cbar = plt.colorbar(fraction=0.032, pad=0.04)
cbar.ax.set_ylim(0,1.4)
plt.show()

In [ ]:
data = np.abs((MK_NLFull-MK_NL7)*cutout).T
data[~cutout.T] = np.nan
plt.imshow(data,cmap='Reds',vmin=0,vmax=2)
plt.axis('off')
cbar = plt.colorbar(ticks=[0,1,2])
cbar.formatter.set_powerlimits((0, 0))
plt.show()

dat = np.abs((AK_SBIFull-AK_SBI7)*cutout).T
data[~cutout.T] = np.nan
norm = TwoSlopeNorm(vmin=0, vcenter=np.nanmax(dat)/2, vmax=np.nanmax(dat))
data = np.abs((AK_NLFull-AK_NL7)*cutout).T
data[~cutout.T] = np.nan
plt.imshow(data,cmap='Reds',norm=norm)
plt.axis('off')
cbar = plt.colorbar(ticks=ticks)
cbar.formatter.set_powerlimits((0, 0))
plt.show()

norm = TwoSlopeNorm(vmin=0, vcenter=1,vmax=2)
data = np.abs((RK_NLFull-RK_NL7)*cutout).T
data[~cutout.T] = np.nan
#ticks = [0, np.round(np.max(data),10)]  #Adjust the number of ticks as needed
plt.imshow(data,cmap='Reds',norm=norm)
plt.axis('off')
cbar = plt.colorbar()
cbar.formatter.set_powerlimits((0, 0))
plt.show()

## c

In [ ]:
i = 1
bvalloc = HCP_dir + 'Pat'+str(i)+'/bvals_1k.txt'
bvecloc = HCP_dir + 'Pat'+str(i)+'/bvecs_1k.txt'
bvalsHCP = np.loadtxt(bvalloc)
bvecsHCP = np.loadtxt(bvecloc)
gtabHCP = gradient_table(bvalsHCP, bvecsHCP)

# Choose the first point (arbitrary starting point, e.g., the first gradient)
selected_indices = [1]
distance_matrix = squareform(pdist(bvecsHCP))
# Iteratively select the point furthest from the current selection
for _ in range(5):  # We need 7 points in total, and one is already selected
    remaining_indices = list(set(range(len(bvecsHCP))) - set(selected_indices))
    
    # Calculate the minimum distance to the selected points for each remaining point
    min_distances = np.min(distance_matrix[remaining_indices][:, selected_indices], axis=1)
    
    # Select the point with the maximum minimum distance
    next_index = remaining_indices[np.argmax(min_distances)]
    selected_indices.append(next_index)

selected_indices7 = [0]+selected_indices

bvalsHCP7_1 = bvalsHCP[selected_indices7]
bvecsHCP7_1 = bvecsHCP[selected_indices7]

bvalloc = HCP_dir + 'Pat'+str(i)+'/bvals_3k.txt'
bvecloc = HCP_dir + 'Pat'+str(i)+'/bvecs_3k.txt'

bvalsHCP3 = np.loadtxt(bvalloc)
bvecsHCP3 = np.loadtxt(bvecloc)
gtabHCP3 = gradient_table(bvalsHCP, bvecsHCP)

# Choose the first point (arbitrary starting point, e.g., the first gradient)
selected_indices = [0]

temp_bvecs = bvecsHCP3[bvalsHCP3>0]
temp_bvals = bvalsHCP3[bvalsHCP3>0]
distance_matrix = squareform(pdist(temp_bvecs))
# Iteratively select the point furthest from the current selection
for _ in range(14):  # We need 7 points in total, and one is already selected
    remaining_indices = list(set(range(len(temp_bvecs))) - set(selected_indices))
    
    # Calculate the minimum distance to the selected points for each remaining point
    min_distances = np.min(distance_matrix[remaining_indices][:, selected_indices], axis=1)
    
    # Select the point with the maximum minimum distance
    next_index = remaining_indices[np.argmax(min_distances)]
    selected_indices.append(next_index)

bvalsHCP7_3 = temp_bvals[selected_indices]
bvecsHCP7_3 = temp_bvecs[selected_indices]

gtabHCP7 = gradient_table(np.hstack((bvalsHCP7_1,bvalsHCP7_3)), np.vstack((bvecsHCP7_1,bvecsHCP7_3)))

true_indx = []
for b in bvecsHCP7_3:
    true_indx.append(np.linalg.norm(b-bvecsHCP3,axis=1).argmin())
selected_indices7 = selected_indices7+[t+69 for t in true_indx]

# Choose the first point (arbitrary starting point, e.g., the first gradient)
selected_indices = [1]
distance_matrix = squareform(pdist(bvecsHCP))

temp_bvecs = bvecsHCP[bvalsHCP>0]
temp_bvals = bvalsHCP[bvalsHCP>0]
distance_matrix = squareform(pdist(temp_bvecs))
# Iteratively select the point furthest from the current selection
for _ in range(18):  # We need 7 points in total, and one is already selected
    remaining_indices = list(set(range(len(temp_bvecs))) - set(selected_indices))
    
    # Calculate the minimum distance to the selected points for each remaining point
    min_distances = np.min(distance_matrix[remaining_indices][:, selected_indices], axis=1)
    
    # Select the point with the maximum minimum distance
    next_index = remaining_indices[np.argmax(min_distances)]
    selected_indices.append(next_index)

temp = selected_indices

bvalsHCP7_1 = np.insert(temp_bvals[temp],0,0)
bvecsHCP7_1 = np.insert(temp_bvecs[temp],0,[0,0,0],axis=0)

bvalloc = HCP_dir + 'Pat'+str(i)+'/bvals_3k.txt'
bvecloc = HCP_dir + 'Pat'+str(i)+'/bvecs_3k.txt'

bvalsHCP3 = np.loadtxt(bvalloc)
bvecsHCP3 = np.loadtxt(bvecloc)
gtabHCP3 = gradient_table(bvalsHCP, bvecsHCP)

# Choose the first point (arbitrary starting point, e.g., the first gradient)
selected_indices = [0]

temp_bvecs = bvecsHCP3[bvalsHCP3>0]
temp_bvals = bvalsHCP3[bvalsHCP3>0]
distance_matrix = squareform(pdist(temp_bvecs))
# Iteratively select the point furthest from the current selection
for _ in range(27):  # We need 7 points in total, and one is already selected
    remaining_indices = list(set(range(len(temp_bvecs))) - set(selected_indices))
    
    # Calculate the minimum distance to the selected points for each remaining point
    min_distances = np.min(distance_matrix[remaining_indices][:, selected_indices], axis=1)
    
    # Select the point with the maximum minimum distance
    next_index = remaining_indices[np.argmax(min_distances)]
    selected_indices.append(next_index)

bvalsHCP7_3 = temp_bvals[selected_indices]
bvecsHCP7_3 = temp_bvecs[selected_indices]

gtabHCP20 = gradient_table(np.hstack((bvalsHCP7_1,bvalsHCP7_3)), np.vstack((bvecsHCP7_1,bvecsHCP7_3)))

true_indx_one = []
for b in bvecsHCP7_1:
    true_indx_one.append(np.linalg.norm(b-bvecsHCP,axis=1).argmin())
true_indx = []        
for b in bvecsHCP7_3:
    true_indx.append(np.linalg.norm(b-bvecsHCP3,axis=1).argmin())
selected_indices20 = true_indx_one+[t+69 for t in true_indx]

In [ ]:
gTabsF = []
gTabs7 = []
gTabs20 = []

FullDat   = []

for i in tqdm(range(1,33)):
    fdwi = HCP_dir + 'Pat'+str(i)+'/diff_1k.nii.gz'
    bvalloc = HCP_dir + 'Pat'+str(i)+'/bvals_1k.txt'
    bvecloc = HCP_dir + 'Pat'+str(i)+'/bvecs_1k.txt'
    
    fdwi3 = HCP_dir + 'Pat'+str(i)+'/diff_3k.nii.gz'
    bvalloc3 = HCP_dir + 'Pat'+str(i)+'/bvals_3k.txt'
    bvecloc3 = HCP_dir + 'Pat'+str(i)+'/bvecs_3k.txt'
    
    bvalsHCP = np.loadtxt(bvalloc)
    bvecsHCP = np.loadtxt(bvecloc)
    gtabHCP = gradient_table(bvalsHCP, bvecsHCP)
    
    bvalsHCP3 = np.loadtxt(bvalloc3)
    bvecsHCP3 = np.loadtxt(bvecloc3)
    gtabHCP3 = gradient_table(bvalsHCP3, bvecsHCP3)
    
    gtabExt  = gradient_table(np.hstack((bvalsHCP,bvalsHCP3)), np.vstack((bvecsHCP,bvecsHCP3)))
    gTabsF.append(gtabExt)
    
    bvalsHCP7 = gtabExt.bvals[selected_indices7]
    bvecsHCP7 = gtabExt.bvecs[selected_indices7]
    gtabHCP7 = gradient_table(bvalsHCP7, bvecsHCP7)
    gTabs7.append(gtabHCP7)

    bvalsHCP20 = gtabExt.bvals[selected_indices20]
    bvecsHCP20 = gtabExt.bvecs[selected_indices20]
    gtabHCP20 = gradient_table(bvalsHCP20, bvecsHCP20)
    gTabs20.append(gtabHCP20)

In [ ]:
TD = []
axial_middles = []
masks = []
WMs = []
for kk in tqdm(range(32)):
    fdwi = HCP_dir + 'Pat'+str(kk+1)+'/diff_1k.nii.gz'
    bvalloc = HCP_dir + 'Pat'+str(kk+1)+'/bvals_1k.txt'
    bvecloc = HCP_dir + 'Pat'+str(kk+1)+'/bvecs_1k.txt'
    
    fdwi3 = HCP_dir + 'Pat'+str(kk+1)+'/diff_3k.nii.gz'
    bvalloc3 = HCP_dir + 'Pat'+str(kk+1)+'/bvals_3k.txt'
    bvecloc3 = HCP_dir + 'Pat'+str(kk+1)+'/bvecs_3k.txt'
    
    bvalsHCP = np.loadtxt(bvalloc)
    bvecsHCP = np.loadtxt(bvecloc)
    gtabHCP = gradient_table(bvalsHCP, bvecsHCP)
    
    bvalsHCP3 = np.loadtxt(bvalloc3)
    bvecsHCP3 = np.loadtxt(bvecloc3)
    gtabHCP3 = gradient_table(bvalsHCP3, bvecsHCP3)
    
    gtabExt  = gradient_table(bvals = np.hstack((bvalsHCP,bvalsHCP3)),bvecs =  np.vstack((bvecsHCP,bvecsHCP3)))
    
    data, affine, img = load_nifti(fdwi, return_img=True)
    data, affine = reslice(data, affine, (1.5,1.5,1.5), (2.5,2.5,2.5))
    maskdata, mask = median_otsu(data, vol_idx=range(10, 50), median_radius=3,
                                 numpass=1, autocrop=False, dilate=2)
    _, mask2 = median_otsu(data, vol_idx=range(10, 50), median_radius=3,
                                 numpass=1, autocrop=True, dilate=2)
    
    
    data3, affine, img = load_nifti(fdwi3, return_img=True)
    data3, affine = reslice(data3, affine, (1.5,1.5,1.5), (2.5,2.5,2.5))
    # Get the indices of True values
    true_indices = np.argwhere(mask)
    
    # Determine the minimum and maximum indices along each dimension
    min_coords = true_indices.min(axis=0)
    max_coords = true_indices.max(axis=0)
    
    maskdata  = maskdata[min_coords[0]:max_coords[0]+1,min_coords[1]:max_coords[1]+1,min_coords[2]:max_coords[2]+1]
    axial_middle = maskdata.shape[2] // 2
    maskdata3 = data3[min_coords[0]:max_coords[0]+1,min_coords[1]:max_coords[1]+1,min_coords[2]:max_coords[2]+1]
    axial_middles.append(axial_middle)
    TestData = np.concatenate([maskdata[:, :, axial_middle, :],maskdata3[:, :, axial_middle, :]],axis=-1)
    TestData4D = np.concatenate([maskdata,maskdata3],axis=-1)
    floor = np.clip(TestData4D.min(axis=-1),-np.inf,0)
    TestData4D_2 = np.copy(TestData4D)
    TestData4D_2[floor <0 ] = TestData4D[floor < 0] + abs(floor)[floor <0 ,None] + 1e-5
    TD.append(TestData4D_2)
    masks.append(mask[min_coords[0]:max_coords[0]+1,min_coords[1]:max_coords[1]+1,axial_middle])
    WM, affine, img = load_nifti(HCP_dir + 'WM_Masks/c2Pat'+str(kk+1)+'_FP.nii', return_img=True)
    WMs.append(np.fliplr(WM[:,:,axial_middle]>0.8))

In [ ]:
MKFullArr = []
RKFullArr = []
AKFullArr = []
for kk in tqdm(range(32),position = 0,leave = True):

    # Compute the mask where the sum is not zero
    mask = np.sum(TD[kk][:, :, axial_middles[kk], :69], axis=-1) != 0

    # Get the indices where mask is True
    indices = np.argwhere(mask)
    dat = TD[kk]
    Ax = axial_middles[kk]
    gt = gTabsF[kk]
    def optimize_chunk(pixels):
        results = []
        for i, j in pixels:
            posterior_samples_1 = Network.sample((500,), x=DKIFeatures(gt.bvecs,gt.bvals,dat[i,j,Ax, :]),show_progress_bars=False)
            results.append((i, j, posterior_samples_1.mean(axis=0)))
        return results

    chunked_indices = [indices[i:i+ChunkSize] for i in range(0, len(indices), ChunkSize)]
    results = Parallel(n_jobs=8)(
        delayed(optimize_chunk)(chunk) for chunk in tqdm(chunked_indices)
    )

    # Initialize NoiseEst with the appropriate shape
    ArrShape = mask.shape


    NoiseEst = np.zeros(list(ArrShape) + [22])

    # Assign the optimization results to NoiseEst
    for chunk in results:
        for i, j, x in chunk:
            NoiseEst[i, j] = x

    MK_SBIFull  = np.zeros([NoiseEst.shape[0], NoiseEst.shape[1]])
    AK_SBIFull  = np.zeros([NoiseEst.shape[0], NoiseEst.shape[1]])
    RK_SBIFull  = np.zeros([NoiseEst.shape[0], NoiseEst.shape[1]])
    NoiseEst2 =  np.zeros_like(NoiseEst)
    for i in range(NoiseEst.shape[0]):
        for j in range(NoiseEst.shape[1]):    
            NoiseEst2[i,j] = np.hstack([mat_to_vals(clip_negative_eigenvalues(vals_to_mat(NoiseEst[i,j]))),NoiseEst[i,j,6:]])
            Metrics = DKIMetrics(NoiseEst2[i,j][:6],NoiseEst2[i,j][6:21])
            MK_SBIFull[i,j] = Metrics[0]
            AK_SBIFull[i,j] = Metrics[1]
            RK_SBIFull[i,j] = Metrics[2]


    MKFullArr.append(MK_SBIFull)
    RKFullArr.append(RK_SBIFull)
    AKFullArr.append(AK_SBIFull)

In [ ]:
MKMinArr = []
RKMinArr = []
AKMinArr = []
for kk in tqdm(range(32)):

    # Compute the mask where the sum is not zero
    mask = np.sum(TD[kk][:, :, axial_middles[kk], :69], axis=-1) != 0

    # Get the indices where mask is True
    indices = np.argwhere(mask)
    dat = TD[kk]
    Ax = axial_middles[kk]
    gt = gTabs7[kk]
    def optimize_chunk(pixels):
        results = []
        for i, j in pixels:
            posterior_samples_1 = Network.sample((500,), x=DKIFeatures(gt.bvecs,gt.bvals,dat[i,j,Ax, selected_indices7]),show_progress_bars=False)
            results.append((i, j, posterior_samples_1.mean(axis=0)))
        return results

    chunked_indices = [indices[i:i+ChunkSize] for i in range(0, len(indices), ChunkSize)]
    results = Parallel(n_jobs=8)(
        delayed(optimize_chunk)(chunk) for chunk in tqdm(chunked_indices)
    )

    # Initialize NoiseEst with the appropriate shape
    ArrShape = mask.shape


    NoiseEst = np.zeros(list(ArrShape) + [22])

    # Assign the optimization results to NoiseEst
    for chunk in results:
        for i, j, x in chunk:
            NoiseEst[i, j] = x

    NoiseEst2 =  np.zeros_like(NoiseEst)
    for i in range(NoiseEst.shape[0]):
        for j in range(NoiseEst.shape[1]):    
            NoiseEst2[i,j] = np.hstack([mat_to_vals(clip_negative_eigenvalues(vals_to_mat(NoiseEst[i,j]))),NoiseEst[i,j,6:]])

    MK_SBIFull  = np.zeros([NoiseEst.shape[0], NoiseEst.shape[1]])
    AK_SBIFull  = np.zeros([NoiseEst.shape[0], NoiseEst.shape[1]])
    RK_SBIFull  = np.zeros([NoiseEst.shape[0], NoiseEst.shape[1]])

    for i in range(NoiseEst.shape[0]):
        for j in range(NoiseEst.shape[1]): 
            Metrics = DKIMetrics(NoiseEst2[i,j][:6],NoiseEst2[i,j][6:21])
            MK_SBIFull[i,j] = Metrics[0]
            AK_SBIFull[i,j] = Metrics[1]
            RK_SBIFull[i,j] = Metrics[2]


    MKMinArr.append(MK_SBIFull)
    RKMinArr.append(RK_SBIFull)
    AKMinArr.append(AK_SBIFull)

In [ ]:
MKMidArr = []
RKMidArr = []
AKMidArr = []
for kk in tqdm(range(32)):

    # Compute the mask where the sum is not zero
    mask = np.sum(TD[kk][:, :, axial_middles[kk], :69], axis=-1) != 0

    # Get the indices where mask is True
    indices = np.argwhere(mask)
    dat = TD[kk]
    Ax = axial_middles[kk]
    gt = gTabs20[kk]
    def optimize_chunk(pixels):
        results = []
        for i, j in pixels:
            posterior_samples_1 = Network.sample((500,), x=DKIFeatures(gt.bvecs,gt.bvals,dat[i,j,Ax, selected_indices20]),show_progress_bars=False)
            results.append((i, j, posterior_samples_1.mean(axis=0)))
        return results

    chunked_indices = [indices[i:i+ChunkSize] for i in range(0, len(indices), ChunkSize)]
    print('start')
    results = Parallel(n_jobs=8)(
        delayed(optimize_chunk)(chunk) for chunk in tqdm(chunked_indices)
    )
    print('end')
    # Initialize NoiseEst with the appropriate shape
    ArrShape = mask.shape


    NoiseEst = np.zeros(list(ArrShape) + [22])

    # Assign the optimization results to NoiseEst
    for chunk in results:
        for i, j, x in chunk:
            NoiseEst[i, j] = x

    NoiseEst2 =  np.zeros_like(NoiseEst)
    for i in range(NoiseEst.shape[0]):
        for j in range(NoiseEst.shape[1]):    
            NoiseEst2[i,j] = np.hstack([mat_to_vals(clip_negative_eigenvalues(vals_to_mat(NoiseEst[i,j]))),NoiseEst[i,j,6:]])

    MK_SBIFull  = np.zeros([NoiseEst.shape[0], NoiseEst.shape[1]])
    AK_SBIFull  = np.zeros([NoiseEst.shape[0], NoiseEst.shape[1]])
    RK_SBIFull  = np.zeros([NoiseEst.shape[0], NoiseEst.shape[1]])

    for i in range(NoiseEst.shape[0]):
        for j in range(NoiseEst.shape[1]): 
            Metrics = DKIMetrics(NoiseEst2[i,j][:6],NoiseEst2[i,j][6:21])
            MK_SBIFull[i,j] = Metrics[0]
            AK_SBIFull[i,j] = Metrics[1]
            RK_SBIFull[i,j] = Metrics[2]


    MKMidArr.append(MK_SBIFull)
    RKMidArr.append(RK_SBIFull)
    AKMidArr.append(AK_SBIFull)

In [ ]:
MKFullNLArr = []
RKFullNLArr = []
AKFullNLArr = []
MKTFullNLArr = []
KFAFullNLArr = []
for kk in tqdm(range(32),position=0,leave=True):
    dkimodelNL = dki.DiffusionKurtosisModel(gTabsF[kk],fit_method='NLLS')
    dkifitNL = dkimodelNL.fit(TD[kk][:,:,axial_middles[kk]])
    MK_NL7_t  = dkifitNL.mk()
    AK_NL7_t  = dkifitNL.ak()
    RK_NL7_t = dkifitNL.rk()
    MKFullNLArr.append(MK_NL7_t)
    RKFullNLArr.append(RK_NL7_t)
    AKFullNLArr.append(AK_NL7_t)

MKMidNLArr = []
RKMidNLArr = []
AKMidNLArr = []
for kk in tqdm(range(32),position=0,leave=True):
    dkimodelNL = dki.DiffusionKurtosisModel(gTabs20[kk],fit_method='NLLS')
    dkifitNL = dkimodelNL.fit(TD[kk][:,:,axial_middles[kk],selected_indices20])
    MK_NL7_t  = dkifitNL.mk()
    AK_NL7_t  = dkifitNL.ak()
    RK_NL7_t = dkifitNL.rk()
    MKMidNLArr.append(MK_NL7_t)
    RKMidNLArr.append(RK_NL7_t)
    AKMidNLArr.append(AK_NL7_t)

MKMinNLArr = []
RKMinNLArr = []
AKMinNLArr = []
for kk in tqdm(range(32),position=0,leave=True):
    dkimodelNL = dki.DiffusionKurtosisModel(gTabs7[kk],fit_method='NLLS')
    dkifitNL = dkimodelNL.fit(TD[kk][:,:,axial_middles[kk],selected_indices7])
    MK_NL7_t  = dkifitNL.mk()
    AK_NL7_t  = dkifitNL.ak()
    RK_NL7_t = dkifitNL.rk()
    MKMinNLArr.append(MK_NL7_t)
    RKMinNLArr.append(RK_NL7_t)
    AKMinNLArr.append(AK_NL7_t)

In [ ]:
AccM7_MK = []
AccM20_MK = []
AccMFulls_MK = []

AccM7NL_MK = []
AccM20NL_MK = []

SSIM7_MK = []
SSIM20_MK = []
SSIMFulls_MK = []

SSIM7NL_MK = []
SSIM20NL_MK = []
for i in range(32):
    M7 =MKMinArr[i]
    MF =MKFullArr[i]
    Ma = masks[i]
    AccM7_MK.append(np.median(np.abs(M7-MF)[Ma]))


    M7 =MKMidArr[i]
    MF =MKFullArr[i]
    AccM20_MK.append(np.median(np.abs(M7-MF)[Ma]))


    M7 =MKFullArr[i]
    MF =MKFullNLArr[i]
    AccMFulls_MK.append(np.median(np.abs(M7-MF)[Ma]))

    M7 =MKMinNLArr[i]
    M7[np.isnan(M7)] = 0
    MF =MKFullNLArr[i]
    AccM7NL_MK.append(np.median(np.abs(M7-MF)[Ma]))


    M7 =MKMidNLArr[i]
    M7[np.isnan(M7)] = 0
    MF =MKFullNLArr[i]
    AccM20NL_MK.append(np.nanmean(np.abs(M7-MF)[Ma]))

    NS1 =MKMinArr[i]
    NS1 = gaussian_filter(NS1, sigma=0.5)
    NS2 =MKFullArr[i]
    Ma = masks[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7,dat_range=10)
    SSIM7_MK.append(result)

    NS1 =MKMidArr[i]
    NS1 = gaussian_filter(NS1, sigma=0.5)
    NS2 =MKFullArr[i]
    Ma = masks[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7,dat_range=10)
    SSIM20_MK.append(result)
    
    NS1 =MKFullArr[i]
    NS2 =MKFullNLArr[i]
    Ma = masks[i] * NS2 == 0
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7,dat_range=10)
    SSIMFulls_MK.append(result)

    NS1 =MKMinNLArr[i]
    NS1 = gaussian_filter(NS1, sigma=0.5)
    NS2 =MKFullNLArr[i]
    Ma = masks[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7,dat_range=10)
    SSIM7NL_MK.append(result)

    NS1 =MKMidNLArr[i]
    NS1 = gaussian_filter(NS1, sigma=0.5)
    NS2 =MKFullNLArr[i]
    Ma = masks[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7,dat_range=10)
    SSIM20NL_MK.append(result)


Prec7_SBI_MK = []
Prec20_SBI_MK = []
PrecFull_SBI_MK = []

Prec7_NLLS_MK = []
Prec20_NLLS_MK = []
PrecFull_NLLS_MK = []
for i in range(32):
    Prec7_SBI_MK.append(np.std(MKMinArr[i][WMs[i]]))
    Prec20_SBI_MK.append(np.std(MKMidArr[i][WMs[i]]))
    PrecFull_SBI_MK.append(np.std(MKFullArr[i][WMs[i]]))

    Prec7_NLLS_MK.append(np.std(MKMinNLArr[i][WMs[i]]))
    Prec20_NLLS_MK.append(np.std(MKMidNLArr[i][WMs[i]]))
    PrecFull_NLLS_MK.append(np.std(MKFullNLArr[i][WMs[i]]))

AccM7_AK = []
AccM20_AK = []
AccMFulls_AK = []

AccM7NL_AK = []
AccM20NL_AK = []

SSIM7_AK = []
SSIM20_AK = []
SSIMFulls_AK = []

SSIM7NL_AK = []
SSIM20NL_AK = []
for i in range(32):
    M7 =AKMinArr[i]
    MF =AKFullArr[i]
    Ma = masks[i]
    AccM7_AK.append(np.median(np.abs(M7-MF)[Ma]))


    M7 =AKMidArr[i]
    MF =AKFullArr[i]
    AccM20_AK.append(np.median(np.abs(M7-MF)[Ma]))


    M7 =AKFullArr[i]
    MF =AKFullNLArr[i]
    AccMFulls_AK.append(np.median(np.abs(M7-MF)[Ma]))

    M7 =AKMinNLArr[i]
    M7[np.isnan(M7)] = 0
    MF =AKFullNLArr[i]
    AccM7NL_AK.append(np.median(np.abs(M7-MF)[Ma]))


    M7 =AKMidNLArr[i]
    M7[np.isnan(M7)] = 0
    MF =AKFullNLArr[i]
    AccM20NL_AK.append(np.nanmean(np.abs(M7-MF)[Ma]))

    
    NS1 =AKMinArr[i]
    NS1 = gaussian_filter(NS1, sigma=0.5)
    NS2 =AKFullArr[i]
    NS2 = gaussian_filter(NS2, sigma=0.5)
    Ma = masks[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7,dat_range=10)
    SSIM7_AK.append(result)

    NS1 =AKMidArr[i]
    NS1 = gaussian_filter(NS1, sigma=0.5)
    NS2 =AKFullArr[i]
    Ma = masks[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7,dat_range=10)
    SSIM20_AK.append(result)
    
    NS1 =AKFullArr[i]
    NS2 =AKFullNLArr[i]
    Ma = masks[i] * NS2 == 0
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7,dat_range=10)
    SSIMFulls_AK.append(result)

    NS1 =AKMinNLArr[i]
    NS1 = gaussian_filter(NS1, sigma=0.5)
    NS2 =AKFullNLArr[i]
    NS2 = gaussian_filter(NS2, sigma=0.5)
    Ma = masks[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7,dat_range=10)
    SSIM7NL_AK.append(result)

    NS1 =AKMidNLArr[i]
    NS1 = gaussian_filter(NS1, sigma=0.5)
    NS2 =AKFullNLArr[i]
    Ma = masks[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7,dat_range=10)
    SSIM20NL_AK.append(result)

Prec7_SBI_AK = []
Prec20_SBI_AK = []
PrecFull_SBI_AK = []

Prec7_NLLS_AK = []
Prec20_NLLS_AK = []
PrecFull_NLLS_AK = []
for i in range(32):
    Prec7_SBI_AK.append(np.std(AKMinArr[i][WMs[i]]))
    Prec20_SBI_AK.append(np.std(AKMidArr[i][WMs[i]]))
    PrecFull_SBI_AK.append(np.std(AKFullArr[i][WMs[i]]))

    Prec7_NLLS_AK.append(np.std(AKMinNLArr[i][WMs[i]]))
    Prec20_NLLS_AK.append(np.std(AKMidNLArr[i][WMs[i]]))
    PrecFull_NLLS_AK.append(np.std(AKFullNLArr[i][WMs[i]]))


AccM7_RK = []
AccM20_RK = []
AccMFulls_RK = []

AccM7NL_RK = []
AccM20NL_RK = []

SSIM7_RK = []
SSIM20_RK = []
SSIMFulls_RK = []

SSIM7NL_RK = []
SSIM20NL_RK = []
for i in range(32):
    M7 =RKMinArr[i]
    MF =RKFullArr[i]
    Ma = masks[i]
    AccM7_RK.append(np.median(np.abs(M7-MF)[Ma]))


    M7 =RKMidArr[i]
    MF =RKFullArr[i]
    AccM20_RK.append(np.median(np.abs(M7-MF)[Ma]))


    M7 =RKFullArr[i]
    MF =RKFullNLArr[i]
    AccMFulls_RK.append(np.median(np.abs(M7-MF)[Ma]))

    M7 =RKMinNLArr[i]
    M7[np.isnan(M7)] = 0
    MF =RKFullNLArr[i]
    AccM7NL_RK.append(np.median(np.abs(M7-MF)[Ma]))


    M7 =RKMidNLArr[i]
    M7[np.isnan(M7)] = 0
    MF =RKFullNLArr[i]
    AccM20NL_RK.append(np.nanmean(np.abs(M7-MF)[Ma]))

    
    NS1 =RKMinArr[i]
    NS1 = gaussian_filter(NS1, sigma=0.5)
    NS2 =RKFullArr[i]
    Ma = masks[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7,dat_range=10)
    SSIM7_RK.append(result)

    NS1 =RKMidArr[i]
    NS1 = gaussian_filter(NS1, sigma=0.5)
    NS2 =RKFullArr[i]
    Ma = masks[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7,dat_range=10)
    SSIM20_RK.append(result)
    
    NS1 =RKFullArr[i]
    NS2 =RKFullNLArr[i]
    Ma = masks[i] * NS2 == 0
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7,dat_range=10)
    SSIMFulls_RK.append(result)

    NS1 =RKMinNLArr[i]
    NS1 = gaussian_filter(NS1, sigma=0.5)
    NS2 =RKFullNLArr[i]
    Ma = masks[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7,dat_range=10)
    SSIM7NL_RK.append(result)

    NS1 =RKMidNLArr[i]
    NS1 = gaussian_filter(NS1, sigma=0.5)
    NS2 =RKFullNLArr[i]
    Ma = masks[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7,dat_range=10)
    SSIM20NL_RK.append(result)

Prec7_SBI_RK = []
Prec20_SBI_RK = []
PrecFull_SBI_RK = []

Prec7_NLLS_RK = []
Prec20_NLLS_RK = []
PrecFull_NLLS_RK = []
for i in range(32):
    Prec7_SBI_RK.append(np.std(RKMinArr[i][WMs[i]]))
    Prec20_SBI_RK.append(np.std(RKMidArr[i][WMs[i]]))
    PrecFull_SBI_RK.append(np.std(RKFullArr[i][WMs[i]]))

    Prec7_NLLS_RK.append(np.std(RKMinNLArr[i][WMs[i]]))
    Prec20_NLLS_RK.append(np.std(RKMidNLArr[i][WMs[i]]))
    PrecFull_NLLS_RK.append(np.std(RKFullNLArr[i][WMs[i]]))

    

In [ ]:
MSDir = '../../MS_data/'

In [ ]:
gTabsF = []
Dats   = []

gTabs7 = []
gTabs20 = []
Masks = []
WMDir = MSDir+'WM_masks/'
WMs = []
for i,Name in tqdm(enumerate(['NMSS_11_1year','NMSS_15','NMSS_16','NMSS_18','NMSS_19','Ctrl055_R01_28','Ctrl056_R01_29','Ctrl057_R01_30'])):
    MatDir = MSDir+Name

    F = pmt.read_mat(MatDir+'/data_loaded.mat')
    affine = np.ones((4,4))
    
    data, affine = reslice(F['data'], affine, (2,2,2), (2.5,2.5,2.5))
    _, maskCut = median_otsu(data, vol_idx=range(10, 50),autocrop=False)

    true_indices = np.argwhere(maskCut)
    
    # Determine the minimum and maximum indices along each dimension
    min_coords = true_indices.min(axis=0)
    max_coords = true_indices.max(axis=0)
    
    for k,x in enumerate(os.listdir(WMDir)):
        if Name in x:
            print(Name)
            WM, affine, img = load_nifti(WMDir+x, return_img=True)
            WM, affine = reslice(WM, affine, (2,2,2), (2.5,2.5,2.5))
            if(i<5):
                WM_t = np.fliplr(np.swapaxes(WM,0,1))
            else:
                WM_t = np.fliplr(np.flipud(np.swapaxes(WM,0,1)))
            WM_t  = WM_t[min_coords[0]:max_coords[0]+1,min_coords[1]:max_coords[1]+1,min_coords[2]:max_coords[2]+1]
            WMs.append(WM_t)
            
    maskdata, mask = median_otsu(data, vol_idx=range(10, 50),autocrop=True)
    axial_middle = maskdata.shape[2] // 2
    Masks.append(mask)
    bvecs = (F['direction'].T/np.linalg.norm(F['direction'],axis=1)).T
    bvecs[np.isnan(bvecs)] = 0
    bvals = F['bval']
    bvecs2000 = bvecs[bvals==2000]
    bvecs4000 = bvecs[bvals==4000]

    bvals2000 = np.array([0] + list(bvals[bvals==2000]))
    bvecs2000 = np.vstack([[0,0,0],bvecs[bvals==2000]])

    Dats.append(maskdata[:,:,:,:])
    
    gTabsF.append(gradient_table(bvals,bvecs))
    if i == 0:
        # Choose the first point (arbitrary starting point, e.g., the first gradient)
        selected_indices = [0]
        distance_matrix = squareform(pdist(bvecs2000))
        # Iteratively select the point furthest from the current selection
        for _ in range(6):  # We need 7 points in total, and one is already selected
            remaining_indices = list(set(range(len(bvecs2000))) - set(selected_indices))
            
            # Calculate the minimum distance to the selected points for each remaining point
            min_distances = np.min(distance_matrix[remaining_indices][:, selected_indices], axis=1)
            
            # Select the point with the maximum minimum distance
            next_index = remaining_indices[np.argmax(min_distances)]
            selected_indices.append(next_index)
        
        selected_indices7 = selected_indices
        
        # Choose the first point (arbitrary starting point, e.g., the first gradient)
        selected_indices = [0]
        distance_matrix = squareform(pdist(bvecs4000))
        # Iteratively select the point furthest from the current selection
        for _ in range(14):  # We need 7 points in total, and one is already selected
            remaining_indices = list(set(range(len(bvecs4000))) - set(selected_indices))
            
            # Calculate the minimum distance to the selected points for each remaining point
            min_distances = np.min(distance_matrix[remaining_indices][:, selected_indices], axis=1)
            
            # Select the point with the maximum minimum distance
            next_index = remaining_indices[np.argmax(min_distances)]
            selected_indices.append(next_index)
        selected_indices7_2 = selected_indices
        
        selected_indices = [0]
        distance_matrix = squareform(pdist(bvecs2000))
        # Iteratively select the point furthest from the current selection
        for _ in range(19):  # We need 7 points in total, and one is already selected
            remaining_indices = list(set(range(len(bvecs2000))) - set(selected_indices))
            
            # Calculate the minimum distance to the selected points for each remaining point
            min_distances = np.min(distance_matrix[remaining_indices][:, selected_indices], axis=1)
            
            # Select the point with the maximum minimum distance
            next_index = remaining_indices[np.argmax(min_distances)]
            selected_indices.append(next_index)
        selected_indices20 = selected_indices
        
        # Choose the first point (arbitrary starting point, e.g., the first gradient)
        selected_indices = [0]
        distance_matrix = squareform(pdist(bvecs4000))
        # Iteratively select the point furthest from the current selection
        for _ in range(27):  # We need 7 points in total, and one is already selected
            remaining_indices = list(set(range(len(bvecs4000))) - set(selected_indices))
            
            # Calculate the minimum distance to the selected points for each remaining point
            min_distances = np.min(distance_matrix[remaining_indices][:, selected_indices], axis=1)
            
            # Select the point with the maximum minimum distance
            next_index = remaining_indices[np.argmax(min_distances)]
            selected_indices.append(next_index)
        selected_indices20_2 = selected_indices
    
        Indxs7 = np.hstack([0,np.where(bvals==2000)[0][np.array(selected_indices7)[1:]-1],np.where(bvals==4000)[0][selected_indices7_2]])
        Indxs20 = np.hstack([0,np.where(bvals==2000)[0][np.array(selected_indices20)[1:]-1],np.where(bvals==4000)[0][selected_indices20_2]])
    gTabs7.append(gradient_table([0]+[2000]*6 + [4000]*15,np.vstack([bvecs2000[selected_indices7],bvecs4000[selected_indices7_2]])))
    gTabs20.append(gradient_table([0]+[2000]*19 + [4000]*28,np.vstack([bvecs2000[selected_indices20],bvecs4000[selected_indices20_2]])))
axial_middles = [32]*8

In [ ]:
MKFullArr = []
RKFullArr = []
AKFullArr = []
MKTFullArr = []
KFAFullArr = []
for kk in range(8):
    Dat = Dats[kk]
    gt = gTabsF[kk]
    ArrShape = Dat.shape[:2]
    AM = axial_middles[kk]
    # Compute the mask where the sum is not zero
    mask = np.sum(Dat[:, :, AM, :], axis=-1) != 0
    
    # Get the indices where mask is True
    indices = np.argwhere(mask)
    def optimize_chunk(pixels):
        results = []
        for i, j in pixels:
            posterior_samples_1 = Network.sample((500,), x=DKIFeatures(gt.bvecs,gt.bvals,Dat[i,j,AM, :]),show_progress_bars=False)
            results.append((i, j, posterior_samples_1.mean(axis=0)))
        return results
    
    chunked_indices = [indices[i:i+ChunkSize] for i in range(0, len(indices), ChunkSize)]
    results = Parallel(n_jobs=8)(
        delayed(optimize_chunk)(chunk) for chunk in tqdm(chunked_indices)
    )
    
    # Initialize NoiseEst with the appropriate shape
    ArrShape = mask.shape
    
    NoiseEst = np.zeros(list(ArrShape) + [22])
    
    # Assign the optimization results to NoiseEst
    for chunk in results:
        for i, j, x in chunk:
            NoiseEst[i, j] = x
        
    NoiseEst2 =  np.zeros_like(NoiseEst)
    for i in range(ArrShape[0]):
        for j in range(ArrShape[1]):  
            NoiseEst2[i,j] = np.hstack([mat_to_vals(clip_negative_eigenvalues(vals_to_mat(NoiseEst[i,j]))),NoiseEst[i,j,6:]])
    
    MK7  = np.zeros(ArrShape)
    RK7  = np.zeros(ArrShape)
    AK7  = np.zeros(ArrShape)
    MKT7 = np.zeros(ArrShape)
    KFA7 = np.zeros(ArrShape)
    for i in tqdm(range(ArrShape[0]),position=0,leave=True):
        for j in range(ArrShape[1]):
            Metrics = DKIMetrics(NoiseEst2[i,j][:6],NoiseEst2[i,j][6:21])
            MK7[i,j] = Metrics[0]
            AK7[i,j] = Metrics[1]
            RK7[i,j] = Metrics[2]
            MKT7[i,j] = Metrics[3]
            KFA7[i,j] = Metrics[4]
    MKFullArr.append(MK7)
    RKFullArr.append(RK7)
    AKFullArr.append(AK7)
    MKTFullArr.append(MKT7)
    KFAFullArr.append(KFA7)
    fig,ax = plt.subplots(1,5,figsize=(24,6.4))
    plt.sca(ax[0])
    plt.imshow(MK7.T,vmin=0,vmax=1)
    plt.sca(ax[1])
    plt.imshow(RK7.T,vmin=0,vmax=1)
    plt.sca(ax[2])
    plt.imshow(AK7.T,vmin=0,vmax=1)
    plt.sca(ax[3])
    plt.imshow(MKT7.T,vmin=0,vmax=1)
    plt.sca(ax[4])
    plt.imshow(KFA7.T,vmin=0,vmax=1)
    plt.show()

In [ ]:
MKMidArr = []
RKMidArr = []
AKMidArr = []
MKTMidArr = []
KFAMidArr = []
for kk in range(8):
    Dat = Dats[kk]
    ArrShape = Dat.shape[:2]
    
    # Compute the mask where the sum is not zero
    mask = np.sum(Dats[kk][:, :, axial_middles[kk], :], axis=-1) != 0
    
    # Get the indices where mask is True
    indices = np.argwhere(mask)
    Arr = Dats[kk][:,:,axial_middles[kk], Indxs20]
    def optimize_chunk(pixels):
        results = []
        for i, j in pixels:
            posterior_samples_1 = Network.sample((500,), x=DKIFeatures(gt.bvecs[Indxs20],gt.bvals[Indxs20],Arr[i,j]),show_progress_bars=False)
            results.append((i, j, posterior_samples_1.mean(axis=0)))
        return results
    
    chunked_indices = [indices[i:i+ChunkSize] for i in range(0, len(indices), ChunkSize)]
    results = Parallel(n_jobs=8)(
        delayed(optimize_chunk)(chunk) for chunk in tqdm(chunked_indices)
    )
    
    NoiseEst = np.zeros(list(ArrShape) + [22])
    
    # Assign the optimization results to NoiseEst
    for chunk in results:
        for i, j, x in chunk:
            NoiseEst[i, j] = x
    NoiseEst2 =  np.zeros_like(NoiseEst)
    for i in range(ArrShape[0]):
        for j in range(ArrShape[1]):  
            NoiseEst2[i,j] = np.hstack([mat_to_vals(clip_negative_eigenvalues(vals_to_mat(NoiseEst[i,j]))),NoiseEst[i,j,6:]])
    
    MK7  = np.zeros(ArrShape)
    RK7  = np.zeros(ArrShape)
    AK7  = np.zeros(ArrShape)
    MKT7 = np.zeros(ArrShape)
    KFA7 = np.zeros(ArrShape)
    for i in tqdm(range(ArrShape[0]),position=0,leave=True):
        for j in range(ArrShape[1]):
            Metrics = DKIMetrics(NoiseEst2[i,j][:6],NoiseEst2[i,j][6:21])
            MK7[i,j] = Metrics[0]
            AK7[i,j] = Metrics[1]
            RK7[i,j] = Metrics[2]
            MKT7[i,j] = Metrics[3]
            KFA7[i,j] = Metrics[4]
    MKMidArr.append(MK7)
    RKMidArr.append(RK7)
    AKMidArr.append(AK7)
    MKTMidArr.append(MKT7)
    KFAMidArr.append(KFA7)
    fig,ax = plt.subplots(1,5,figsize=(24,6.4))
    plt.sca(ax[0])
    plt.imshow(MK7.T,vmin=0,vmax=1)
    plt.sca(ax[1])
    plt.imshow(RK7.T,vmin=0,vmax=1)
    plt.sca(ax[2])
    plt.imshow(AK7.T,vmin=0,vmax=1)
    plt.sca(ax[3])
    plt.imshow(MKT7.T,vmin=0,vmax=1)
    plt.sca(ax[4])
    plt.imshow(KFA7.T,vmin=0,vmax=1)
    plt.show()

In [ ]:
MKMinArr = []
RKMinArr = []
AKMinArr = []
MKTMinArr = []
KFAMinArr = []
for kk in range(8):
    Dat = Dats[kk]
    ArrShape = Dat.shape[:2]
    
    # Compute the mask where the sum is not zero
    mask = np.sum(Dats[kk][:, :, axial_middles[kk], :], axis=-1) != 0
    
    # Get the indices where mask is True
    indices = np.argwhere(mask)
    Arr = Dats[kk][:,:,axial_middles[kk], Indxs7]
    def optimize_chunk(pixels):
        results = []
        for i, j in pixels:
            posterior_samples_1 = Network.sample((500,), x=DKIFeatures(gt.bvecs[Indxs7],gt.bvals[Indxs7],Arr[i,j]),show_progress_bars=False)
            results.append((i, j, posterior_samples_1.mean(axis=0)))
        return results
    
    chunked_indices = [indices[i:i+ChunkSize] for i in range(0, len(indices), ChunkSize)]
    results = Parallel(n_jobs=8)(
        delayed(optimize_chunk)(chunk) for chunk in tqdm(chunked_indices)
    )
    
    NoiseEst = np.zeros(list(ArrShape) + [22])
    
    # Assign the optimization results to NoiseEst
    for chunk in results:
        for i, j, x in chunk:
            NoiseEst[i, j] = x
    NoiseEst2 =  np.zeros_like(NoiseEst)
    for i in range(ArrShape[0]):
        for j in range(ArrShape[1]):  
            NoiseEst2[i,j] = np.hstack([mat_to_vals(clip_negative_eigenvalues(vals_to_mat(NoiseEst[i,j]))),NoiseEst[i,j,6:]])
    
    MK7  = np.zeros(ArrShape)
    RK7  = np.zeros(ArrShape)
    AK7  = np.zeros(ArrShape)
    MKT7 = np.zeros(ArrShape)
    KFA7 = np.zeros(ArrShape)
    for i in tqdm(range(ArrShape[0]),position=0,leave=True):
        for j in range(ArrShape[1]):
            Metrics = DKIMetrics(NoiseEst2[i,j][:6],NoiseEst2[i,j][6:21])
            MK7[i,j] = Metrics[0]
            AK7[i,j] = Metrics[1]
            RK7[i,j] = Metrics[2]
            MKT7[i,j] = Metrics[3]
            KFA7[i,j] = Metrics[4]
    MKMinArr.append(MK7)
    RKMinArr.append(RK7)
    AKMinArr.append(AK7)
    MKTMinArr.append(MKT7)
    KFAMinArr.append(KFA7)
    fig,ax = plt.subplots(1,5,figsize=(24,6.4))
    plt.sca(ax[0])
    plt.imshow(MK7.T,vmin=0,vmax=1)
    plt.sca(ax[1])
    plt.imshow(RK7.T,vmin=0,vmax=1)
    plt.sca(ax[2])
    plt.imshow(AK7.T,vmin=0,vmax=1)
    plt.sca(ax[3])
    plt.imshow(MKT7.T,vmin=0,vmax=1)
    plt.sca(ax[4])
    plt.imshow(KFA7.T,vmin=0,vmax=1)
    plt.show()

In [ ]:
MKFullNLArr = []
RKFullNLArr = []
AKFullNLArr = []
MKTFullNLArr = []
KFAFullNLArr = []
for kk in range(8):
    dkimodelNL = dki.DiffusionKurtosisModel(gTabsF[kk],fit_method='NLLS')
    dkifitNL = dkimodelNL.fit(Dats[kk][:,:,axial_middles[kk],:])
    MK_NLFull  = dkifitNL.mk()
    AK_NLFull  = dkifitNL.ak()
    RK_NLFull = dkifitNL.rk()
    
    MKFullNLArr.append(MK_NLFull)
    RKFullNLArr.append(RK_NLFull)
    AKFullNLArr.append(AK_NLFull)
    #MKTFullNLArr.append(MKT_NLFull)
    #KFAFullNLArr.append(KFA_NLFull)
    
    fig,ax = plt.subplots(1,5,figsize=(24,6.4))
    plt.sca(ax[0])
    plt.imshow(MK_NLFull.T,vmin=0,vmax=1)
    plt.sca(ax[1])
    plt.imshow(RK_NLFull.T,vmin=0,vmax=1)
    plt.sca(ax[2])
    plt.imshow(AK_NLFull.T,vmin=0,vmax=1)
    plt.sca(ax[3])
    #plt.imshow(MKT_NLFull.T,vmin=0,vmax=1)
    plt.sca(ax[4])
    #plt.imshow(KFA_NLFull.T,vmin=0,vmax=1)
    plt.show()

In [ ]:
MKMidNLArr = []
RKMidNLArr = []
AKMidNLArr = []
MKTFullNLArr = []
KFAFullNLArr = []
for kk in range(8):
    dkimodelNL = dki.DiffusionKurtosisModel(gTabs20[kk],fit_method='NLLS')
    dkifitNL = dkimodelNL.fit(Dats[kk][:,:,axial_middles[kk],Indxs20])
    MK_NLFull  = dkifitNL.mk()
    AK_NLFull  = dkifitNL.ak()
    RK_NLFull = dkifitNL.rk()
    
    MKMidNLArr.append(MK_NLFull)
    RKMidNLArr.append(RK_NLFull)
    AKMidNLArr.append(AK_NLFull)
    #MKTFullNLArr.append(MKT_NLFull)
    #KFAFullNLArr.append(KFA_NLFull)
    
    fig,ax = plt.subplots(1,5,figsize=(24,6.4))
    plt.sca(ax[0])
    plt.imshow(MK_NLFull.T,vmin=0,vmax=1)
    plt.sca(ax[1])
    plt.imshow(RK_NLFull.T,vmin=0,vmax=1)
    plt.sca(ax[2])
    plt.imshow(AK_NLFull.T,vmin=0,vmax=1)
    plt.sca(ax[3])
    #plt.imshow(MKT_NLFull.T,vmin=0,vmax=1)
    plt.sca(ax[4])
    #plt.imshow(KFA_NLFull.T,vmin=0,vmax=1)
    plt.show()

In [ ]:
MKMinNLArr = []
RKMinNLArr = []
AKMinNLArr = []
MKTFullNLArr = []
KFAFullNLArr = []
for kk in range(8):
    dkimodelNL = dki.DiffusionKurtosisModel(gTabs7[kk],fit_method='NLLS')
    dkifitNL = dkimodelNL.fit(Dats[kk][:,:,axial_middles[kk],Indxs7])
    MK_NLFull  = dkifitNL.mk()
    AK_NLFull  = dkifitNL.ak()
    RK_NLFull = dkifitNL.rk()
    
    MKMinNLArr.append(MK_NLFull)
    RKMinNLArr.append(RK_NLFull)
    AKMinNLArr.append(AK_NLFull)
    #MKTFullNLArr.append(MKT_NLFull)
    #KFAFullNLArr.append(KFA_NLFull)
    
    fig,ax = plt.subplots(1,5,figsize=(24,6.4))
    plt.sca(ax[0])
    plt.imshow(MK_NLFull.T,vmin=0,vmax=1)
    plt.sca(ax[1])
    plt.imshow(RK_NLFull.T,vmin=0,vmax=1)
    plt.sca(ax[2])
    plt.imshow(AK_NLFull.T,vmin=0,vmax=1)
    plt.sca(ax[3])
    #plt.imshow(MKT_NLFull.T,vmin=0,vmax=1)
    plt.sca(ax[4])
    #plt.imshow(KFA_NLFull.T,vmin=0,vmax=1)
    plt.show()

In [ ]:
AccM7_MK_MS= []
AccM20_MK_MS= []
AccMFulls_MK_MS= []

AccM7NL_MK_MS= []
AccM20NL_MK_MS= []

SSIM7_MK_MS= []
SSIM20_MK_MS= []
SSIMFulls_MK_MS= []

SSIM7NL_MK_MS= []
SSIM20NL_MK_MS= []
for i in range(8):
    M7 = MKMinArr[i]
    MF = MKFullArr[i]
    Ma = Masks[i][:,:,axial_middles[i]]
    AccM7_MK_MS.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = MKMidArr[i]
    MF = MKFullArr[i]
    Ma = Masks[i][:,:,axial_middles[i]]
    AccM20_MK_MS.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = MKFullArr[i]
    MF = MKFullNLArr[i]
    Ma = Masks[i][:,:,axial_middles[i]]
    AccMFulls_MK_MS.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = MKMinNLArr[i]
    MF = MKFullNLArr[i]

    AccM7NL_MK_MS.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = MKMidNLArr[i]
    MF = MKFullNLArr[i]

    AccM20NL_MK_MS.append(np.mean(np.abs(M7-MF)[Ma]))

    NS1 = MKMinArr[i]
    NS2 = MKFullArr[i]

    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM7_MK_MS.append(result)

    NS1 = MKMidArr[i]
    NS2 = MKFullArr[i]

    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM20_MK_MS.append(result)
    
    NS1 = MKFullArr[i]
    NS2 = MKFullNLArr[i]

    result = masked_local_ssim(NS1, NS2, Ma * (abs(NS2) > 0), win_size=7)
    SSIMFulls_MK_MS.append(result)

    NS1 = MKMinNLArr[i]
    NS2 = MKFullNLArr[i]

    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM7NL_MK_MS.append(result)

    NS1 = MKMidNLArr[i]
    NS2 = MKFullNLArr[i]

    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM20NL_MK_MS.append(result)


Prec7_SBI_MK_MS= []
Prec20_SBI_MK_MS= []
PrecFull_SBI_MK_MS= []

Prec7_NLLS_MK_MS= []
Prec20_NLLS_MK_MS= []
PrecFull_NLLS_MK_MS= []
for i in range(8):
    Prec7_SBI_MK_MS.append(np.std(MKMinArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))
    Prec20_SBI_MK_MS.append(np.std(MKMidArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))
    PrecFull_SBI_MK_MS.append(np.std(MKFullArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))

    Prec7_NLLS_MK_MS.append(np.std(MKMinNLArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))
    Prec20_NLLS_MK_MS.append(np.std(MKMidNLArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))
    PrecFull_NLLS_MK_MS.append(np.std(MKFullNLArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))


In [ ]:
AccM7_AK_MS = []
AccM20_AK_MS = []
AccMFulls_AK_MS = []

AccM7NL_AK_MS = []
AccM20NL_AK_MS = []

SSIM7_AK_MS = []
SSIM20_AK_MS = []
SSIMFulls_AK_MS = []

SSIM7NL_AK_MS = []
SSIM20NL_AK_MS = []
for i in range(8):
    M7 = AKMinArr[i]
    MF = AKFullArr[i]
    Ma = Masks[i][:,:,axial_middles[i]]
    AccM7_AK_MS.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = AKMidArr[i]
    MF = AKFullArr[i]
    Ma = Masks[i][:,:,axial_middles[i]]
    AccM20_AK_MS.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = AKFullArr[i]
    MF = AKFullNLArr[i]
    Ma = Masks[i][:,:,axial_middles[i]]
    AccMFulls_AK_MS.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = AKMinNLArr[i]
    MF = AKFullNLArr[i]

    AccM7NL_AK_MS.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = AKMidNLArr[i]
    MF = AKFullNLArr[i]

    AccM20NL_AK_MS.append(np.mean(np.abs(M7-MF)[Ma]))

    NS1 = AKMinArr[i]
    NS2 = AKFullArr[i]

    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM7_AK_MS.append(result)

    NS1 = AKMidArr[i]
    NS2 = AKFullArr[i]

    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM20_AK_MS.append(result)
    
    NS1 = AKFullArr[i]
    NS2 = AKFullNLArr[i]

    result = masked_local_ssim(NS1, NS2, Ma * (abs(NS2) > 0), win_size=7)
    SSIMFulls_AK_MS.append(result)

    NS1 = AKMinNLArr[i]
    NS2 = AKFullNLArr[i]

    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM7NL_AK_MS.append(result)

    NS1 = AKMidNLArr[i]
    NS2 = AKFullNLArr[i]

    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM20NL_AK_MS.append(result)


Prec7_SBI_AK_MS = []
Prec20_SBI_AK_MS = []
PrecFull_SBI_AK_MS = []

Prec7_NLLS_AK_MS = []
Prec20_NLLS_AK_MS = []
PrecFull_NLLS_AK_MS = []
for i in range(8):
    Prec7_SBI_AK_MS.append(np.std(AKMinArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))
    Prec20_SBI_AK_MS.append(np.std(AKMidArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))
    PrecFull_SBI_AK_MS.append(np.std(AKFullArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))

    Prec7_NLLS_AK_MS.append(np.std(AKMinNLArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))
    Prec20_NLLS_AK_MS.append(np.std(AKMidNLArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))
    PrecFull_NLLS_AK_MS.append(np.std(AKFullNLArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))


In [ ]:
AccM7_RK_MS = []
AccM20_RK_MS = []
AccMFulls_RK_MS = []

AccM7NL_RK_MS = []
AccM20NL_RK_MS = []

SSIM7_RK_MS = []
SSIM20_RK_MS = []
SSIMFulls_RK_MS = []

SSIM7NL_RK_MS = []
SSIM20NL_RK_MS = []
for i in range(8):
    M7 = RKMinArr[i]
    MF = RKFullArr[i]
    Ma = Masks[i][:,:,axial_middles[i]]
    AccM7_RK_MS.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = RKMidArr[i]
    MF = RKFullArr[i]
    Ma = Masks[i][:,:,axial_middles[i]]
    AccM20_RK_MS.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = RKFullArr[i]
    MF = RKFullNLArr[i]
    Ma = Masks[i][:,:,axial_middles[i]]
    AccMFulls_RK_MS.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = RKMinNLArr[i]
    MF = RKFullNLArr[i]

    AccM7NL_RK_MS.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = RKMidNLArr[i]
    MF = RKFullNLArr[i]

    AccM20NL_RK_MS.append(np.mean(np.abs(M7-MF)[Ma]))

    NS1 = RKMinArr[i]
    NS2 = RKFullArr[i]

    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM7_RK_MS.append(result)

    NS1 = RKMidArr[i]
    NS2 = RKFullArr[i]

    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM20_RK_MS.append(result)
    
    NS1 = RKFullArr[i]
    NS2 = RKFullNLArr[i]

    result = masked_local_ssim(NS1, NS2, Ma * (abs(NS2) > 0), win_size=7)
    SSIMFulls_RK_MS.append(result)

    NS1 = RKMinNLArr[i]
    NS2 = RKFullNLArr[i]

    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM7NL_RK_MS.append(result)

    NS1 = RKMidNLArr[i]
    NS2 = RKFullNLArr[i]

    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM20NL_RK_MS.append(result)


Prec7_SBI_RK_MS = []
Prec20_SBI_RK_MS = []
PrecFull_SBI_RK_MS = []

Prec7_NLLS_RK_MS = []
Prec20_NLLS_RK_MS = []
PrecFull_NLLS_RK_MS = []
for i in range(8):
    Prec7_SBI_RK_MS.append(np.std(RKMinArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))
    Prec20_SBI_RK_MS.append(np.std(RKMidArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))
    PrecFull_SBI_RK_MS.append(np.std(RKFullArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))

    Prec7_NLLS_RK_MS.append(np.std(RKMinNLArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))
    Prec20_NLLS_RK_MS.append(np.std(RKMidNLArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))
    PrecFull_NLLS_RK_MS.append(np.std(RKFullNLArr[i][WMs[i][:,:,axial_middles[i]]>0.8]))


In [ ]:
MK_RT_SBI= []
AK_RT_SBI = []
RK_RT_SBI = []

MK_RT_SBI_Min = []
AK_RT_SBI_Min = []
RK_RT_SBI_Min = []

MK_RT_NLLS_Min = []
AK_RT_NLLS_Min = []
RK_RT_NLLS_Min = []

MK_RT_NLLS_Full = []
AK_RT_NLLS_Full = []
RK_RT_NLLS_Full = []
for jj,N in enumerate(Names):
    Subfiles = []
    for k,x in enumerate(os.listdir(RetestDir)):
        if N in x:
            print(x)
            Subfiles.append(x)
    Subfiles = sorted(Subfiles)

    S = Subfiles[0]
    MatDir = RetestDir+S
    F = pmt.read_mat(MatDir+'/data_loaded.mat')
    affine1 = np.eye(4)

    data, affine1 = reslice(F['data'], affine1, (2,2,2), (2.5,2.5,2.5))
    _, maskCut = median_otsu(data, vol_idx=range(10, 80), autocrop=False)
    true_indices = np.argwhere(maskCut)

    # Determine the minimum and maximum indices along each dimension
    min_coords = true_indices.min(axis=0)
    max_coords = true_indices.max(axis=0)
    AM = (max_coords[-1]+min_coords[-1])//2
    bvecs = (F['direction'].T/np.linalg.norm(F['direction'],axis=1)).T
    bvecs[np.isnan(bvecs)] = 0
    bvals = F['bval']
    
    bvecs2000 = bvecs[bvals==2000]
    bvecs4000 = bvecs[bvals==4000]
    bvals2000 = np.array([0] + list(bvals[bvals==2000]))
    bvecs2000 = np.vstack([[0,0,0],bvecs[bvals==2000]])
    # Choose the first point (arbitrary starting point, e.g., the first gradient)
    selected_indices = [0]
    distance_matrix = squareform(pdist(bvecs2000))
    # Iteratively select the point furthest from the current selection
    for _ in range(6):  # We need 7 points in total, and one is already selected
        remaining_indices = list(set(range(len(bvecs2000))) - set(selected_indices))

        # Calculate the minimum distance to the selected points for each remaining point
        min_distances = np.min(distance_matrix[remaining_indices][:, selected_indices], axis=1)

        # Select the point with the maximum minimum distance
        next_index = remaining_indices[np.argmax(min_distances)]
        selected_indices.append(next_index)

    selected_indices7 = selected_indices

    # Choose the first point (arbitrary starting point, e.g., the first gradient)
    selected_indices = [0]
    distance_matrix = squareform(pdist(bvecs4000))
    # Iteratively select the point furthest from the current selection
    for _ in range(14):  # We need 7 points in total, and one is already selected
        remaining_indices = list(set(range(len(bvecs4000))) - set(selected_indices))

        # Calculate the minimum distance to the selected points for each remaining point
        min_distances = np.min(distance_matrix[remaining_indices][:, selected_indices], axis=1)

        # Select the point with the maximum minimum distance
        next_index = remaining_indices[np.argmax(min_distances)]
        selected_indices.append(next_index)
    selected_indices7_2 = selected_indices
    Indxs7 = [np.hstack([0,np.where(bvals==2000)[0][np.array(selected_indices7)[1:]-1],np.where(bvals==4000)[0][selected_indices7_2]])]
    gTabs7 = [gradient_table([0]+[2000]*6 + [4000]*15,np.vstack([bvecs2000[selected_indices7],bvecs4000[selected_indices7_2]]))]
    

    gtabs = [gradient_table(bvals = bvals,bvecs = bvecs)]
    Dats = []
    for i,S in enumerate(Subfiles[1:]):
        MatDir = RetestDir+S
        F = pmt.read_mat(MatDir+'/data_loaded.mat')
        affine = np.eye(4)

        data1, affine = reslice(F['data'], affine, (2,2,2), (2.5,2.5,2.5))

        bvecs = (F['direction'].T/np.linalg.norm(F['direction'],axis=1)).T
        bvecs[np.isnan(bvecs)] = 0
        bvals = F['bval']
        if(jj == 0):
            if(i < 2):
                data1 = data1[:,::-1]
        elif(jj == 1 or jj == 2 or jj == 3 or jj == 4):
            if(i<3):
                data1 = data1[:,::-1]
        elif(jj==5):
            if(i>0 and i < 3):
                data1 = data1[:,::-1]
        Dats.append(data1)
        gtabs.append(gradient_table(bvals = bvals,bvecs = bvecs))
        
        bvecs2000 = bvecs[bvals==2000]
        bvecs4000 = bvecs[bvals==4000]
        bvals2000 = np.array([0] + list(bvals[bvals==2000]))
        bvecs2000 = np.vstack([[0,0,0],bvecs[bvals==2000]])
        # Choose the first point (arbitrary starting point, e.g., the first gradient)
        selected_indices = [0]
        distance_matrix = squareform(pdist(bvecs2000))
        # Iteratively select the point furthest from the current selection
        for _ in range(6):  # We need 7 points in total, and one is already selected
            remaining_indices = list(set(range(len(bvecs2000))) - set(selected_indices))

            # Calculate the minimum distance to the selected points for each remaining point
            min_distances = np.min(distance_matrix[remaining_indices][:, selected_indices], axis=1)

            # Select the point with the maximum minimum distance
            next_index = remaining_indices[np.argmax(min_distances)]
            selected_indices.append(next_index)

        selected_indices7 = selected_indices

        # Choose the first point (arbitrary starting point, e.g., the first gradient)
        selected_indices = [0]
        distance_matrix = squareform(pdist(bvecs4000))
        # Iteratively select the point furthest from the current selection
        for _ in range(14):  # We need 7 points in total, and one is already selected
            remaining_indices = list(set(range(len(bvecs4000))) - set(selected_indices))

            # Calculate the minimum distance to the selected points for each remaining point
            min_distances = np.min(distance_matrix[remaining_indices][:, selected_indices], axis=1)

            # Select the point with the maximum minimum distance
            next_index = remaining_indices[np.argmax(min_distances)]
            selected_indices.append(next_index)
        selected_indices7_2 = selected_indices
        Indxs7.append(np.hstack([0,np.where(bvals==2000)[0][np.array(selected_indices7)[1:]-1],np.where(bvals==4000)[0][selected_indices7_2]]))
        gTabs7.append(gradient_table([0]+[2000]*6 + [4000]*15,np.vstack([bvecs2000[selected_indices7],bvecs4000[selected_indices7_2]])))

    NewDats = [data]
    for d,gt in zip(Dats,gtabs[1:]):
        affine_map = rigid_register(data[...,gtabs[0].bvals==0].mean(axis=-1),d[...,gt.bvals==0].mean(axis=-1),affine1,affine1)
        data2_warp = np.array([affine_map.transform(d[:,:,:,i], interpolation="linear") for i in range(len(gt.bvals))])
        data2_warp = np.rollaxis(data2_warp, 0, data2_warp.ndim)
        NewDats.append(data2_warp)
    
    NewDats_masked = [ND*maskCut[...,None] for ND in NewDats]
    
    MK_Arr = []
    AK_Arr = []
    RK_Arr = []
    for ND,gt in tqdm(zip(NewDats_masked,gtabs),position=0,leave=True):
        mask = np.sum(ND[:, :, AM, :], axis=-1) != 0
        # Get the indices where mask is True
        indices = np.argwhere(mask)
        floor = np.clip(ND.min(axis=-1),-np.inf,0)
        dat = ND + abs(floor)[:,:,:,None] + 1e-5
        # Define the function for optimization
        def optimize_chunk(pixels):
            results = []
            for i, j in pixels:
                samples = Network.sample((1000,), x=DKIFeatures(gt.bvecs,gt.bvals,dat[i,j,AM]),show_progress_bars=False)
                results.append((i, j, samples.mean(axis=0)))
            return results

        chunked_indices = [indices[i:i+ChunkSize] for i in range(0, len(indices), ChunkSize)]
        results = Parallel(n_jobs=8)(
            delayed(optimize_chunk)(chunk) for chunk in tqdm(chunked_indices,position=0,leave=True)
        )
        ArrShape = mask.shape
        NoiseEst = np.zeros(list(ArrShape) + [22])

        # Assign the optimization results to NoiseEst
        for chunk in results:
            for i, j, x in chunk:
                NoiseEst[i, j] = x

        NoiseEst2 =  np.zeros_like(NoiseEst)
        for i in range(ArrShape[0]):
            for j in range(ArrShape[1]):  
                NoiseEst2[i,j] = np.hstack([mat_to_vals(clip_negative_eigenvalues(vals_to_mat(NoiseEst[i,j]))),NoiseEst[i,j,6:]])

        MK7  = np.zeros(ArrShape)
        RK7  = np.zeros(ArrShape)
        AK7  = np.zeros(ArrShape)
        for i in tqdm(range(ArrShape[0]),position=0,leave=True):
            for j in range(ArrShape[1]):
                Metrics = DKIMetrics(NoiseEst2[i,j][:6],NoiseEst2[i,j][6:21])
                MK7[i,j] = Metrics[0]
                AK7[i,j] = Metrics[1]
                RK7[i,j] = Metrics[2]
        MK_Arr.append(MK7)
        AK_Arr.append(AK7)
        RK_Arr.append(RK7)
    MK_RT_SBI.append(MK_Arr)
    AK_RT_SBI.append(AK_Arr)
    RK_RT_SBI.append(RK_Arr)

    MK_Arr = []
    AK_Arr = []
    RK_Arr = []
    for ND,gt,idx in tqdm(zip(NewDats_masked,gTabs7,Indxs7),position=0,leave=True):
        mask = np.sum(ND[:, :, AM, :], axis=-1) != 0
        # Get the indices where mask is True
        indices = np.argwhere(mask)
        floor = np.clip(ND.min(axis=-1),-np.inf,0)
        dat = ND + abs(floor)[:,:,:,None] + 1e-5
        dat = dat[...,idx]
        def optimize_chunk(pixels):
            results = []
            for i, j in pixels:
                samples = Network.sample((1000,), x=DKIFeatures(gt.bvecs,gt.bvals,dat[i,j,AM]),show_progress_bars=False)
                results.append((i, j, samples.mean(axis=0)))
            return results

        chunked_indices = [indices[i:i+ChunkSize] for i in range(0, len(indices), ChunkSize)]
        results = Parallel(n_jobs=8)(
            delayed(optimize_chunk)(chunk) for chunk in tqdm(chunked_indices,position=0,leave=True)
        )
        
        NoiseEst = np.zeros(list(ArrShape) + [22])

        # Assign the optimization results to NoiseEst
        for chunk in results:
            for i, j, x in chunk:
                NoiseEst[i, j] = x

        NoiseEst2 =  np.zeros_like(NoiseEst)
        for i in range(ArrShape[0]):
            for j in range(ArrShape[1]):  
                NoiseEst2[i,j] = np.hstack([mat_to_vals(clip_negative_eigenvalues(vals_to_mat(NoiseEst[i,j]))),NoiseEst[i,j,6:]])

        MK7  = np.zeros(ArrShape)
        RK7  = np.zeros(ArrShape)
        AK7  = np.zeros(ArrShape)
        for i in tqdm(range(ArrShape[0]),position=0,leave=True):
            for j in range(ArrShape[1]):
                Metrics = DKIMetrics(NoiseEst2[i,j][:6],NoiseEst2[i,j][6:21])
                MK7[i,j] = Metrics[0]
                AK7[i,j] = Metrics[1]
                RK7[i,j] = Metrics[2]
        MK_Arr.append(MK7)
        AK_Arr.append(AK7)
        RK_Arr.append(RK7)
    MK_RT_SBI_Min.append(MK_Arr)
    AK_RT_SBI_Min_list.append(AK_Arr)
    RK_RT_SBI_Min_list.append(RK_Arr)    
    
    
    MK_NLS_arr = []
    AK_NLS_arr = []
    RK_NLS_arr = []
    for d,gt,idx in zip(NewDats_masked,gTabs7,Indxs7):
        tenmodel = dki.DiffusionKurtosisModel(gt,return_S0_hat = True)
        tenfit = tenmodel.fit(d[:,:,AM,idx])

        MK_NLS_arr.append(tenfit.mk())
        AK_NLS_arr.append(tenfit.ak())
        RK_NLS_arr.append(tenfit.rk())
    
    MK_RT_NLLS_Min.append(MK_NLS_arr)
    AK_RT_NLLS_Min.append(AK_NLS_arr)
    RK_RT_NLLS_Min.append(RK_NLS_arr)
    
    MK_NLS_arr = []
    AK_NLS_arr = []
    RK_NLS_arr = []
    for d,gt in zip(NewDats_masked,gtabs):
        tenmodel = dki.DiffusionKurtosisModel(gt,return_S0_hat = True)
        tenfit = tenmodel.fit(d[:,:,AM])

        MK_NLS_arr.append(tenfit.mk())
        AK_NLS_arr.append(tenfit.ak())
        RK_NLS_arr.append(tenfit.rk())
    
    MK_RT_NLLS_Full.append(MK_NLS_arr)
    AK_RT_NLLS_Full.append(AK_NLS_arr)
    RK_RT_NLLS_Full.append(RK_NLS_arr)

In [ ]:
MK_RT_SBI_Min_list = []
RK_RT_SBI_Min_list = []
AK_RT_SBI_Min_list = []
MK_RT_SBI_Full_list = []
RK_RT_SBI_Full_list = []
AK_RT_SBI_Full_list = []

MK_RT_NLLS_Min_list = []
RK_RT_NLLS_Min_list = []
AK_RT_NLLS_Min_list = []
MK_RT_NLLS_Full_list = []
RK_RT_NLLS_Full_list = []
AK_RT_NLLS_Full_list = []


for kk in range(len(MK_RT_SBI_Min)):
    for i in range(len(MK_RT_SBI_Min[kk])):
        MK_RT_SBI_Min_list.append(MK_RT_SBI_Min[kk][i])
        AK_RT_SBI_Min_list.append(AK_RT_SBI_Min[kk][i])
        RK_RT_SBI_Min_list.append(RK_RT_SBI_Min[kk][i])

        MK_RT_SBI_Full_list.append(MK_RT_SBI_Full[kk][i])
        AK_RT_SBI_Full_list.append(AK_RT_SBI_Full[kk][i])
        RK_RT_SBI_Full_list.append(RK_RT_SBI_Full[kk][i])
        
        MK_RT_NLLS_Min_list.append(MK_RT_NLLS_Min[kk][i])
        AK_RT_NLLS_Min_list.append(AK_RT_NLLS_Min[kk][i])
        RK_RT_NLLS_Min_list.append(RK_RT_NLLS_Min[kk][i])

        MK_RT_NLLS_Full_list.append(MK_RT_NLLS_Full[kk][i])
        AK_RT_NLLS_Full_list.append(AK_RT_NLLS_Full[kk][i])
        RK_RT_NLLS_Full_list.append(RK_RT_NLLS_Full[kk][i])

In [ ]:
AccM7_MK_RT = []
AccMFulls_MK_RT = []

AccM7NL_MK_RT = []

SSIM7_MK_RT = []
SSIMFulls_MK_RT = []

SSIM7NL_MK_RT = []
for i in tqdm(range(29),position=0,leave=True):
    M7 = MK_RT_SBI_Min_list[i]
    MF = MK_RT_SBI_Full_list[i]
    Ma = np.logical_not(MK_RT_SBI_Min_list[i]==0)
    AccM7_MK_RT.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = MK_RT_SBI_Full_list[i]
    MF = MK_RT_NLLS_Full_list[i]
    AccMFulls_MK_RT.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = MK_RT_NLLS_Min_list[i]
    MF = MK_RT_NLLS_Full_list[i]
    AccM7NL_MK_RT.append(np.mean(np.abs(M7-MF)[Ma]))


    NS1 = MK_RT_SBI_Min_list[i]
    NS2 = MK_RT_SBI_Full_list[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM7_MK_RT.append(result)

    NS1 = MK_RT_SBI_Full_list[i]
    NS2 = MK_RT_NLLS_Full_list[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIMFulls_MK_RT.append(result)

    NS1 = MK_RT_NLLS_Min_list[i]
    NS2 = MK_RT_NLLS_Full_list[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM7NL_MK_RT.append(result)



In [ ]:
AccM7_AK_RT = []
AccMFulls_AK_RT = []

AccM7NL_AK_RT = []

SSIM7_AK_RT = []
SSIMFulls_AK_RT = []

SSIM7NL_AK_RT = []
for i in tqdm(range(29),position=0,leave=True):
    M7 = AK_RT_SBI_Min_list[i]
    MF = AK_RT_SBI_Full_list[i]
    Ma = np.logical_not(MK_RT_SBI_Min_list[i]==0)
    AccM7_AK_RT.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = AK_RT_SBI_Full_list[i]
    MF = AK_RT_NLLS_Full_list[i]
    AccMFulls_AK_RT.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = AK_RT_NLLS_Min_list[i]
    MF = AK_RT_NLLS_Full_list[i]
    AccM7NL_AK_RT.append(np.mean(np.abs(M7-MF)[Ma]))


    NS1 = AK_RT_SBI_Min_list[i]
    NS2 = AK_RT_SBI_Full_list[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM7_AK_RT.append(result)

    NS1 = AK_RT_SBI_Full_list[i]
    NS2 = AK_RT_NLLS_Full_list[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIMFulls_AK_RT.append(result)

    NS1 = AK_RT_NLLS_Min_list[i]
    NS2 = AK_RT_NLLS_Full_list[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM7NL_AK_RT.append(result)



In [ ]:
AccM7_RK_RT = []
AccMFulls_RK_RT = []

AccM7NL_RK_RT = []

SSIM7_RK_RT = []
SSIMFulls_RK_RT = []

SSIM7NL_RK_RT = []
for i in tqdm(range(29),position=0,leave=True):
    M7 = RK_RT_SBI_Min_list[i]
    MF = RK_RT_SBI_Full_list[i]
    Ma = np.logical_not(MK_RT_SBI_Min_list[i]==0)
    AccM7_RK_RT.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = RK_RT_SBI_Full_list[i]
    MF = RK_RT_NLLS_Full_list[i]
    AccMFulls_RK_RT.append(np.mean(np.abs(M7-MF)[Ma]))

    M7 = RK_RT_NLLS_Min_list[i]
    MF = RK_RT_NLLS_Full_list[i]
    AccM7NL_RK_RT.append(np.mean(np.abs(M7-MF)[Ma]))


    NS1 = RK_RT_SBI_Min_list[i]
    NS2 = RK_RT_SBI_Full_list[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM7_RK_RT.append(result)

    NS1 = RK_RT_SBI_Full_list[i]
    NS2 = RK_RT_NLLS_Full_list[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIMFulls_RK_RT.append(result)

    NS1 = RK_RT_NLLS_Min_list[i]
    NS2 = RK_RT_NLLS_Full_list[i]
    result = masked_local_ssim(NS1, NS2, Ma, win_size=7)
    SSIM7NL_RK_RT.append(result)



In [ ]:
# Plot setup
fig, ax2 = plt.subplots(figsize=(3.2,4.8))#, sharex=True)
fig.subplots_adjust(hspace=0.05)

y_data = np.array(AccM7NL_MK + AccM7NL_MK_MS+AccM7NL_MK_RT)
g_pos = np.array([3.05])
colors = ['burlywood']
colors2 = ['peachpuff']

BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=False)
plt.xticks([1,1.7,2,2.8,3.1],['Full','Mid','Min','Mid','Min'],fontsize=32,rotation=90)
#plt.gca().ticklabel_format(axis='y',style='sci',scilimits=(-1,1))
plt.yticks(fontsize=24)

y_data = np.array(AccM7NL_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
ax2.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.8)

y_data = np.array(AccM7NL_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
ax2.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
ax2.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

y_data = np.array(AccM7NL_MK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
ax2.scatter(x_data,y_data,marker='s',color='chocolate',s=100,alpha=0.5)


plt.sca(ax2)

y_data = np.array(AccMFulls_MK + AccMFulls_MK_MS+AccMFulls_MK_RT)
g_pos = np.array([0.8])
colors = ['black']
colors2 = ['gray']

BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=False)

y_data = np.array(AccMFulls_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color='gray',s=100,alpha=0.5)

y_data = np.array(AccMFulls_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='black',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='black',s=100,alpha=0.5)

y_data = np.array(AccMFulls_MK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
ax2.scatter(x_data,y_data,marker='s',color='gray',s=100,alpha=0.5)

y_data = np.array(AccM20_MK + AccM20_MK_MS)
g_pos = np.array([1.55])
colors = ['lightseagreen']
colors2 = ['paleturquoise']

BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=False)

y_data = np.array(AccM20_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5,label='HPC')

y_data = np.array(AccM20_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

y_data = np.array(AccM7_MK + AccM7_MK_MS+AccM7_MK_RT)
g_pos = np.array([1.95])
colors = ['mediumturquoise']
colors2 = ['paleturquoise']

BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=False)

y_data = np.array(AccM7_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)


y_data = np.array(AccM7_MK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
ax2.scatter(x_data,y_data,marker='s',color='mediumaquamarine',s=100,alpha=0.5,label='GH\ndata')

y_data = np.array(AccM7_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

g_pos = np.array([2.65])
colors = ['sandybrown']
colors2 = ['peachpuff']

y_data = np.array(AccM20NL_MK + AccM20NL_MK_MS)

BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=False)

y_data = np.array(AccM20NL_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(AccM20NL_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

y_data = np.array(AccM20NL_MK + AccM20NL_MK_MS)

plt.xticks([0.8,1.55,1.95,2.65,3.05],['Full','Mid','Min','Mid','Min'],fontsize=32,rotation=90)
plt.gca().ticklabel_format(axis='y',style='sci',scilimits=(-0.5,3))
plt.yticks(fontsize=24)

#ax2.set_ylim(0, 0.00016)

# Common x-ticks

# Adding broken axis effect
d = .5
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12, linestyle="none", color='k', mec='k', mew=1, clip_on=False)

leg = plt.legend(
    loc='upper left',         # base location  # fine-tune the legend's position
    frameon=False, ncols=1,
fontsize=32,columnspacing=0.3,handlelength=0.6,handletextpad=0.3,bbox_to_anchor= (-0.05,1.0),markerscale=1.5)

for h in leg.legend_handles:
    try:
        h.set_alpha(1)
    except AttributeError:
        pass

In [ ]:
fig, ax1 = plt.subplots(figsize=(3.2,4.8))#, sharex=True)
fig.subplots_adjust(hspace=0.05)  # adjust space between Axes

y_data = np.array(SSIMFulls_MK + SSIMFulls_MK_MS+SSIMFulls_MK_RT)
g_pos = np.array([0.8])
colors = ['black']
colors2 = ['gray']

BoxPlots(y_data,g_pos,colors,colors2,ax1,widths=0.2,scatter=False)

y_data = np.array(SSIMFulls_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color='gray',s=100,alpha=0.5)

y_data = np.array(SSIMFulls_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='black',s=100,alpha=0.7)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='black',s=100,alpha=0.7)

y_data = np.array(SSIMFulls_MK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color='gray',s=100,alpha=0.5)

y_data = np.array(SSIM20_MK+SSIM20_MK_MS)
g_pos = np.array([1.55])
colors = ['lightseagreen']
colors2 = ['paleturquoise']

BoxPlots(y_data,g_pos,colors,colors2,ax1,widths=0.2,scatter=False,scatter_alpha=0.5)

y_data = np.array(SSIM20_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(SSIM20_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5,label='MS-\nlesions')
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5,label='MS-\nctrl')




y_data = np.array(SSIM7_MK+SSIM7_MK_MS+SSIM7_MK_RT)
g_pos = np.array([1.95])
colors = ['mediumturquoise']
colors2 = ['paleturquoise']

BoxPlots(y_data,g_pos,colors,colors2,ax1,widths=0.2,scatter=True)

y_data = np.array(SSIM7_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(SSIM7_MK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color='mediumaquamarine',s=100,alpha=0.5)

y_data = np.array(SSIM7_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

y_data = np.array(SSIM20NL_MK +SSIM20NL_MK_MS)
g_pos = np.array([2.65])
colors = ['sandybrown']
colors2 = ['peachpuff']

BoxPlots(y_data,g_pos,colors,colors2,ax1,widths=0.2,scatter=True)

y_data = np.array(SSIM20NL_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(SSIM20NL_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

y_data = np.array(SSIM7NL_MK+SSIM7NL_MK_MS+SSIM7NL_MK_RT)
g_pos = np.array([3.05])
colors = ['burlywood']
colors2 = ['peachpuff']

BoxPlots(y_data,g_pos,colors,colors2,ax1,widths=0.2,scatter=True)
BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=True)

y_data = np.array(SSIM7NL_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(SSIM7NL_MK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='s',color='chocolate',s=100,alpha=0.5)


y_data = np.array(SSIM7NL_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)


plt.xticks([0.8,1.55,1.95,2.65,3.05],['Full','Mid','Min','Mid','Min'],fontsize=32,rotation=90)
plt.gca().ticklabel_format(axis='y',style='sci',scilimits=(-0.5,3))
plt.yticks(fontsize=24)

leg = plt.legend(
    loc='lower left',         # base location  # fine-tune the legend's position
    frameon=False, ncols=1,
fontsize=32,columnspacing=0.3,handlelength=0.6,handletextpad=0.3,markerscale=1.5,bbox_to_anchor=(-0.1,0))

for h in leg.legend_handles:
    try:
        h.set_alpha(1)
    except AttributeError:
        pass
plt.yticks([0,0.2,0.4,0.6,0.8,1])

In [ ]:
fig, ax = plt.subplots(figsize=(3.2,4.8))#, sharex=True)
y_data = np.array(PrecFull_SBI_MK+PrecFull_SBI_MK_MS)
g_pos = np.array([0.65])
colors = ['mediumturquoise']
colors2 = ['paleturquoise']
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=False)

x = np.arange(0.55,1.5,0.05)
y1 = np.ones_like(x)*np.percentile(y_data[~np.isnan(y_data)], 25)
y2 = np.ones_like(x)*np.percentile(y_data[~np.isnan(y_data)], 77)
plt.fill_between(x,y1,y2,color=SBIFit,zorder=10,alpha=0.2,hatch='//')

y_data = np.array(PrecFull_SBI_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(PrecFull_SBI_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

y_data = np.array(Prec20_SBI_MK+Prec20_SBI_MK_MS)
g_pos = np.array([1])
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=False)

y_data = np.array(Prec20_SBI_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(Prec20_SBI_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

y_data = np.array(Prec7_SBI_MK+Prec7_SBI_MK_MS)
g_pos = np.array([1.35])
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=False)

y_data = np.array(Prec7_SBI_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(Prec7_SBI_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

colors = ['sandybrown']
colors2 = ['peachpuff']
y_data = np.array(PrecFull_NLLS_MK+PrecFull_NLLS_MK_MS)
g_pos = np.array([1.8])
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=False)

x = np.arange(1.7,2.6,0.05)
y1 = np.ones_like(x)*np.percentile(np.array(PrecFull_NLLS_MK)[~np.isnan(PrecFull_NLLS_MK)], 25)
y2 = np.ones_like(x)*np.percentile(np.array(PrecFull_NLLS_MK)[~np.isnan(PrecFull_NLLS_MK)], 77)
plt.fill_between(x,y1,y2,color=WLSFit,zorder=10,alpha=0.2,hatch='//')#

y_data = np.array(PrecFull_NLLS_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)


y_data = np.array(PrecFull_NLLS_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

y_data = np.array(Prec20_NLLS_MK)
g_pos = np.array([2.15])
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)

y_data = np.array(Prec20_NLLS_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)


y_data = np.array(Prec20_NLLS_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

y_data = np.array(Prec7_NLLS_MK+Prec7_NLLS_MK_MS)
g_pos = np.array([2.5])
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=False)

y_data = np.array(Prec7_NLLS_MK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)


y_data = np.array(Prec7_NLLS_MK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

plt.xticks([0.65,1,1.35,1.8,2.15,2.5],['Full','Mid','Min','Full','Mid','Min'],fontsize=32,rotation=90)
plt.gca().ticklabel_format(axis='y',style='sci',scilimits=(-0.5,3))
plt.yticks(fontsize=24)
ax.set_xlim([0.3,2.7])
ax.set_yticks([0.2,0.4,0.6,0.8,1])

## d

In [ ]:
# Plot setup
fig, ax2 = plt.subplots(figsize=(3.2,4.8))#, sharex=True)
fig.subplots_adjust(hspace=0.05)

y_data = np.array(AccM7NL_AK + AccM7NL_AK_MS+AccM7NL_AK_RT)
g_pos = np.array([3.05])
colors = ['burlywood']
colors2 = ['peachpuff']

BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=False)
plt.xticks([1,1.7,2,2.8,3.1],['Full','Mid','Min','Mid','Min'],fontsize=32,rotation=90)
#plt.gca().ticklabel_format(axis='y',style='sci',scilimits=(-1,1))
plt.yticks(fontsize=24)

y_data = np.array(AccM7NL_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
ax2.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.8)

y_data = np.array(AccM7NL_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
ax2.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
ax2.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

y_data = np.array(AccM7NL_AK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
ax2.scatter(x_data,y_data,marker='s',color='chocolate',s=100,alpha=0.5)


plt.sca(ax2)

y_data = np.array(AccMFulls_AK + AccMFulls_AK_MS+AccMFulls_AK_RT)
g_pos = np.array([0.8])
colors = ['black']
colors2 = ['gray']

BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=False)

y_data = np.array(AccMFulls_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color='gray',s=100,alpha=0.5)

y_data = np.array(AccMFulls_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='black',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='black',s=100,alpha=0.5)

y_data = np.array(AccMFulls_AK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
ax2.scatter(x_data,y_data,marker='s',color='gray',s=100,alpha=0.5)

y_data = np.array(AccM20_AK + AccM20_AK_MS)
g_pos = np.array([1.55])
colors = ['lightseagreen']
colors2 = ['paleturquoise']

BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=False)

y_data = np.array(AccM20_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5,label='HPC')

y_data = np.array(AccM20_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

y_data = np.array(AccM7_AK + AccM7_AK_MS+AccM7_AK_RT)
g_pos = np.array([1.95])
colors = ['mediumturquoise']
colors2 = ['paleturquoise']

BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=False)

y_data = np.array(AccM7_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)


y_data = np.array(AccM7_AK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
ax2.scatter(x_data,y_data,marker='s',color='mediumaquamarine',s=100,alpha=0.5,label='GH\ndata')

y_data = np.array(AccM7_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

g_pos = np.array([2.65])
colors = ['sandybrown']
colors2 = ['peachpuff']

y_data = np.array(AccM20NL_AK + AccM20NL_AK_MS)

BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=False)

y_data = np.array(AccM20NL_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(AccM20NL_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

y_data = np.array(AccM20NL_AK + AccM20NL_AK_MS)

plt.xticks([0.8,1.55,1.95,2.65,3.05],['Full','Mid','Min','Mid','Min'],fontsize=32,rotation=90)
plt.gca().ticklabel_format(axis='y',style='sci',scilimits=(-0.5,3))
plt.yticks(fontsize=24)

#ax2.set_ylim(0, 0.00016)

# Common x-ticks

# Adding broken axis effect
d = .5
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12, linestyle="none", color='k', mec='k', mew=1, clip_on=False)

if Save: plt.savefig(FigLoc+'DKI_AK_Acc.pdf',format='pdf',bbox_inches='tight',transparent=True)

In [ ]:
fig, ax1 = plt.subplots(figsize=(3.2,4.8))#, sharex=True)
fig.subplots_adjust(hspace=0.05)  # adjust space between Axes

y_data = np.array(SSIMFulls_AK + SSIMFulls_AK_MS+SSIMFulls_AK_RT)
g_pos = np.array([0.8])
colors = ['black']
colors2 = ['gray']

BoxPlots(y_data,g_pos,colors,colors2,ax1,widths=0.2,scatter=False)

y_data = np.array(SSIMFulls_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color='gray',s=100,alpha=0.5)

y_data = np.array(SSIMFulls_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='black',s=100,alpha=0.7)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='black',s=100,alpha=0.7)

y_data = np.array(SSIMFulls_AK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color='gray',s=100,alpha=0.5)

y_data = np.array(SSIM20_AK+SSIM20_AK_MS)
g_pos = np.array([1.55])
colors = ['lightseagreen']
colors2 = ['paleturquoise']

BoxPlots(y_data,g_pos,colors,colors2,ax1,widths=0.2,scatter=False,scatter_alpha=0.5)

y_data = np.array(SSIM20_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(SSIM20_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5,label='MS-\nlesions')
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5,label='MS-\nctrl')




y_data = np.array(SSIM7_AK+SSIM7_AK_MS+SSIM7_AK_RT)
g_pos = np.array([1.95])
colors = ['mediumturquoise']
colors2 = ['paleturquoise']

BoxPlots(y_data,g_pos,colors,colors2,ax1,widths=0.2,scatter=True)

y_data = np.array(SSIM7_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(SSIM7_AK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color='mediumaquamarine',s=100,alpha=0.5)

y_data = np.array(SSIM7_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

y_data = np.array(SSIM20NL_AK +SSIM20NL_AK_MS)
g_pos = np.array([2.65])
colors = ['sandybrown']
colors2 = ['peachpuff']

BoxPlots(y_data,g_pos,colors,colors2,ax1,widths=0.2,scatter=True)

y_data = np.array(SSIM20NL_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(SSIM20NL_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

y_data = np.array(SSIM7NL_AK+SSIM7NL_AK_MS+SSIM7NL_AK_RT)
g_pos = np.array([3.05])
colors = ['burlywood']
colors2 = ['peachpuff']

BoxPlots(y_data,g_pos,colors,colors2,ax1,widths=0.2,scatter=True)
BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=True)

y_data = np.array(SSIM7NL_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(SSIM7NL_AK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='s',color='chocolate',s=100,alpha=0.5)


y_data = np.array(SSIM7NL_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)


plt.xticks([0.8,1.55,1.95,2.65,3.05],['Full','Mid','Min','Mid','Min'],fontsize=32,rotation=90)
plt.gca().ticklabel_format(axis='y',style='sci',scilimits=(-0.5,3))
plt.yticks(fontsize=24)

plt.yticks([0,0.2,0.4,0.6,0.8,1])
if Save: plt.savefig(FigLoc+'DKI_AK_SSIM.pdf',format='pdf',bbox_inches='tight',transparent=True)

In [ ]:
fig, ax = plt.subplots(figsize=(3.2,4.8))#, sharex=True)
y_data = np.array(PrecFull_SBI_AK+PrecFull_SBI_AK_MS)
g_pos = np.array([0.65])
colors = ['mediumturquoise']
colors2 = ['paleturquoise']
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=False)

x = np.arange(0.55,1.5,0.05)
y1 = np.ones_like(x)*np.percentile(y_data[~np.isnan(y_data)], 25)
y2 = np.ones_like(x)*np.percentile(y_data[~np.isnan(y_data)], 77)
plt.fill_between(x,y1,y2,color=SBIFit,zorder=10,alpha=0.2,hatch='//')

y_data = np.array(PrecFull_SBI_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(PrecFull_SBI_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

y_data = np.array(Prec20_SBI_AK+Prec20_SBI_AK_MS)
g_pos = np.array([1])
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=False)

y_data = np.array(Prec20_SBI_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(Prec20_SBI_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

y_data = np.array(Prec7_SBI_AK+Prec7_SBI_AK_MS)
g_pos = np.array([1.35])
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=False)

y_data = np.array(Prec7_SBI_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(Prec7_SBI_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

colors = ['sandybrown']
colors2 = ['peachpuff']
y_data = np.array(PrecFull_NLLS_AK+PrecFull_NLLS_AK_MS)
g_pos = np.array([1.8])
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=False)

x = np.arange(1.7,2.6,0.05)
y1 = np.ones_like(x)*np.percentile(np.array(PrecFull_NLLS_AK)[~np.isnan(PrecFull_NLLS_AK)], 25)
y2 = np.ones_like(x)*np.percentile(np.array(PrecFull_NLLS_AK)[~np.isnan(PrecFull_NLLS_AK)], 77)
plt.fill_between(x,y1,y2,color=WLSFit,zorder=10,alpha=0.2,hatch='//')#

y_data = np.array(PrecFull_NLLS_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)


y_data = np.array(PrecFull_NLLS_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

y_data = np.array(Prec20_NLLS_AK)
g_pos = np.array([2.15])
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)

y_data = np.array(Prec20_NLLS_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)


y_data = np.array(Prec20_NLLS_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

y_data = np.array(Prec7_NLLS_AK+Prec7_NLLS_AK_MS)
g_pos = np.array([2.5])
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=False)

y_data = np.array(Prec7_NLLS_AK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)


y_data = np.array(Prec7_NLLS_AK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

plt.xticks([0.65,1,1.35,1.8,2.15,2.5],['Full','Mid','Min','Full','Mid','Min'],fontsize=32,rotation=90)
plt.gca().ticklabel_format(axis='y',style='sci',scilimits=(-0.5,3))
plt.yticks(fontsize=24)
ax.set_xlim([0.3,2.7])
if Save: plt.savefig(FigLoc+'DKI_AK_Prec.pdf',format='pdf',bbox_inches='tight',transparent=True)

## e

In [ ]:
# Plot setup
fig, ax2 = plt.subplots(figsize=(3.2,4.8))#, sharex=True)
fig.subplots_adjust(hspace=0.05)

y_data = np.array(AccM7NL_RK + AccM7NL_RK_MS+AccM7NL_RK_RT)
g_pos = np.array([3.05])
colors = ['burlywood']
colors2 = ['peachpuff']

BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=False)
plt.xticks([1,1.7,2,2.8,3.1],['Full','Mid','Min','Mid','Min'],fontsize=32,rotation=90)
#plt.gca().ticklabel_format(axis='y',style='sci',scilimits=(-1,1))
plt.yticks(fontsize=24)

y_data = np.array(AccM7NL_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
ax2.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.8)

y_data = np.array(AccM7NL_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
ax2.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
ax2.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

y_data = np.array(AccM7NL_RK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
ax2.scatter(x_data,y_data,marker='s',color='chocolate',s=100,alpha=0.5)


plt.sca(ax2)

y_data = np.array(AccMFulls_RK + AccMFulls_RK_MS+AccMFulls_RK_RT)
g_pos = np.array([0.8])
colors = ['black']
colors2 = ['gray']

BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=False)

y_data = np.array(AccMFulls_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color='gray',s=100,alpha=0.5)

y_data = np.array(AccMFulls_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='black',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='black',s=100,alpha=0.5)

y_data = np.array(AccMFulls_RK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
ax2.scatter(x_data,y_data,marker='s',color='gray',s=100,alpha=0.5)

y_data = np.array(AccM20_RK + AccM20_RK_MS)
g_pos = np.array([1.55])
colors = ['lightseagreen']
colors2 = ['paleturquoise']

BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=False)

y_data = np.array(AccM20_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5,label='HPC')

y_data = np.array(AccM20_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

y_data = np.array(AccM7_RK + AccM7_RK_MS+AccM7_RK_RT)
g_pos = np.array([1.95])
colors = ['mediumturquoise']
colors2 = ['paleturquoise']

BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=False)

y_data = np.array(AccM7_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)


y_data = np.array(AccM7_RK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
ax2.scatter(x_data,y_data,marker='s',color='mediumaquamarine',s=100,alpha=0.5,label='GH\ndata')

y_data = np.array(AccM7_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

g_pos = np.array([2.65])
colors = ['sandybrown']
colors2 = ['peachpuff']

y_data = np.array(AccM20NL_RK + AccM20NL_RK_MS)

BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=False)

y_data = np.array(AccM20NL_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(AccM20NL_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

y_data = np.array(AccM20NL_RK + AccM20NL_RK_MS)

plt.xticks([0.8,1.55,1.95,2.65,3.05],['Full','Mid','Min','Mid','Min'],fontsize=32,rotation=90)
plt.gca().ticklabel_format(axis='y',style='sci',scilimits=(-0.5,3))
plt.yticks(fontsize=24)

#ax2.set_ylim(0, 0.00016)

# Common x-ticks

# Adding broken axis effect
d = .5
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12, linestyle="none", color='k', mec='k', mew=1, clip_on=False)

if Save: plt.savefig(FigLoc+'DKI_RK_Acc.pdf',format='pdf',bbox_inches='tight',transparent=True)

In [ ]:
fig, ax1 = plt.subplots(figsize=(3.2,4.8))#, sharex=True)
fig.subplots_adjust(hspace=0.05)  # adjust space between Axes

y_data = np.array(SSIMFulls_RK + SSIMFulls_RK_MS+SSIMFulls_RK_RT)
g_pos = np.array([0.8])
colors = ['black']
colors2 = ['gray']

BoxPlots(y_data,g_pos,colors,colors2,ax1,widths=0.2,scatter=False)

y_data = np.array(SSIMFulls_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color='gray',s=100,alpha=0.5)

y_data = np.array(SSIMFulls_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='black',s=100,alpha=0.7)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='black',s=100,alpha=0.7)

y_data = np.array(SSIMFulls_RK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color='gray',s=100,alpha=0.5)

y_data = np.array(SSIM20_RK+SSIM20_RK_MS)
g_pos = np.array([1.55])
colors = ['lightseagreen']
colors2 = ['paleturquoise']

BoxPlots(y_data,g_pos,colors,colors2,ax1,widths=0.2,scatter=False,scatter_alpha=0.5)

y_data = np.array(SSIM20_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(SSIM20_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5,label='MS-\nlesions')
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5,label='MS-\nctrl')




y_data = np.array(SSIM7_RK+SSIM7_RK_MS+SSIM7_RK_RT)
g_pos = np.array([1.95])
colors = ['mediumturquoise']
colors2 = ['paleturquoise']

BoxPlots(y_data,g_pos,colors,colors2,ax1,widths=0.2,scatter=True)

y_data = np.array(SSIM7_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(SSIM7_RK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color='mediumaquamarine',s=100,alpha=0.5)

y_data = np.array(SSIM7_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

y_data = np.array(SSIM20NL_RK +SSIM20NL_RK_MS)
g_pos = np.array([2.65])
colors = ['sandybrown']
colors2 = ['peachpuff']

BoxPlots(y_data,g_pos,colors,colors2,ax1,widths=0.2,scatter=True)

y_data = np.array(SSIM20NL_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(SSIM20NL_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

y_data = np.array(SSIM7NL_RK+SSIM7NL_RK_MS+SSIM7NL_RK_RT)
g_pos = np.array([3.05])
colors = ['burlywood']
colors2 = ['peachpuff']

BoxPlots(y_data,g_pos,colors,colors2,ax1,widths=0.2,scatter=True)
BoxPlots(y_data,g_pos,colors,colors2,ax2,widths=0.2,scatter=True)

y_data = np.array(SSIM7NL_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(SSIM7NL_RK_RT)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='s',color='chocolate',s=100,alpha=0.5)


y_data = np.array(SSIM7NL_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)


plt.xticks([0.8,1.55,1.95,2.65,3.05],['Full','Mid','Min','Mid','Min'],fontsize=32,rotation=90)
plt.gca().ticklabel_format(axis='y',style='sci',scilimits=(-0.5,3))
plt.yticks(fontsize=24)
    
plt.yticks([0,0.2,0.4,0.6,0.8,1])
if Save: plt.savefig(FigLoc+'DKI_RK_SSIM.pdf',format='pdf',bbox_inches='tight',transparent=True)

In [ ]:
fig, ax = plt.subplots(figsize=(3.2,4.8))#, sharex=True)
y_data = np.array(PrecFull_SBI_RK+PrecFull_SBI_RK_MS)
g_pos = np.array([0.65])
colors = ['mediumturquoise']
colors2 = ['paleturquoise']
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=False)

x = np.arange(0.55,1.5,0.05)
y1 = np.ones_like(x)*np.percentile(y_data[~np.isnan(y_data)], 25)
y2 = np.ones_like(x)*np.percentile(y_data[~np.isnan(y_data)], 77)
plt.fill_between(x,y1,y2,color=SBIFit,zorder=10,alpha=0.2,hatch='//')

y_data = np.array(PrecFull_SBI_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(PrecFull_SBI_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

y_data = np.array(Prec20_SBI_RK+Prec20_SBI_RK_MS)
g_pos = np.array([1])
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=False)

y_data = np.array(Prec20_SBI_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(Prec20_SBI_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

y_data = np.array(Prec7_SBI_RK+Prec7_SBI_RK_MS)
g_pos = np.array([1.35])
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=False)

y_data = np.array(Prec7_SBI_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)

y_data = np.array(Prec7_SBI_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkcyan',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkcyan',s=100,alpha=0.5)

colors = ['sandybrown']
colors2 = ['peachpuff']
y_data = np.array(PrecFull_NLLS_RK+PrecFull_NLLS_RK_MS)
g_pos = np.array([1.8])
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=False)

x = np.arange(1.7,2.6,0.05)
y1 = np.ones_like(x)*np.percentile(np.array(PrecFull_NLLS_RK)[~np.isnan(PrecFull_NLLS_RK)], 25)
y2 = np.ones_like(x)*np.percentile(np.array(PrecFull_NLLS_RK)[~np.isnan(PrecFull_NLLS_RK)], 77)
plt.fill_between(x,y1,y2,color=WLSFit,zorder=10,alpha=0.2,hatch='//')#

y_data = np.array(PrecFull_NLLS_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)


y_data = np.array(PrecFull_NLLS_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

y_data = np.array(Prec20_NLLS_RK)
g_pos = np.array([2.15])
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)

y_data = np.array(Prec20_NLLS_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)


y_data = np.array(Prec20_NLLS_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

y_data = np.array(Prec7_NLLS_RK+Prec7_NLLS_RK_MS)
g_pos = np.array([2.5])
BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=False)

y_data = np.array(Prec7_NLLS_RK)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data,y_data,marker='o',color=colors2,s=100,alpha=0.5)


y_data = np.array(Prec7_NLLS_RK_MS)
x_data = g_pos*np.ones_like(y_data)
x_data += stats.t(df=6, scale=0.02).rvs(len(x_data))
plt.scatter(x_data[:5],y_data[:5],marker='o',color='darkorange',s=100,alpha=0.5)
plt.scatter(x_data[5:],y_data[5:],marker='^',color='darkorange',s=100,alpha=0.5)

plt.xticks([0.65,1,1.35,1.8,2.15,2.5],['Full','Mid','Min','Full','Mid','Min'],fontsize=32,rotation=90)
plt.gca().ticklabel_format(axis='y',style='sci',scilimits=(-0.5,3))
plt.yticks(fontsize=24)
ax.set_xlim([0.3,2.7])
ax.set_ylim([-0.01,1.5])
if Save: plt.savefig(FigLoc+'DKI_RK_Prec.pdf',format='pdf',bbox_inches='tight',transparent=True)

## f

In [ ]:
Name = 'NMSS_11_1year'
with open(MSDir+Name+'/Les_dict.pkl', 'rb') as f:
    LesDict = pickle.load(f)
_, maskCut = median_otsu(LesDict['data'][...,0], vol_idx=range(10, 50),autocrop=False)

temp = np.copy(LesDict['data'][...,0])
temp[~maskCut] = math.nan
plt.imshow(temp,cmap='gray')
lesion_cmap = ListedColormap(['red'])
contra_cmap = ListedColormap(['lightsalmon'])
plt.imshow(LesDict['lesion'],cmap=lesion_cmap,alpha=0.7)
plt.imshow(LesDict['contra'],cmap=contra_cmap,alpha=0.7)
plt.axis('off')


## g-i

In [ ]:
MK_arr = []
AK_arr = []
RK_arr = []
FA_arr = []
MD_arr = []
for i,Name in tqdm(enumerate(['NMSS_11_1year','NMSS_15','NMSS_16','NMSS_18','NMSS_19'])):
    with open(MSDir+Name+'/Les_dict.pkl', 'rb') as f:
        LesDict = pickle.load(f)
    plt.imshow(LesDict['data'][...,0],cmap='gray')
    plt.imshow(LesDict['lesion'],cmap='autumn',alpha=0.5)
    plt.imshow(LesDict['contra'],cmap='winter',alpha=0.5)
    plt.show()
    plt.imshow(LesDict['data'][...,0],cmap='gray')
    plt.imshow(LesDict['lesion'],cmap='autumn',alpha=0.5)
    plt.imshow(np.fliplr(LesDict['contra']),cmap='winter',alpha=0.5)
    plt.show()
    plt.imshow(LesDict['lesion'],cmap='autumn')
    plt.imshow(LesDict['contra'],cmap='winter')
    plt.imshow(LesDict['contra']*LesDict['lesion'],cmap='gray')
    plt.show()
    bin_les = LesDict['lesion'] == LesDict['lesion']
    bin_con = LesDict['contra'] == LesDict['contra']
    overlap = bin_les*bin_con
    bin_les[overlap] = 0
    bin_con[overlap] = 0

    MatDir = MSDir+Name

    F = pmt.read_mat(MatDir+'/data_loaded.mat')
    bvecs = (F['direction'].T/np.linalg.norm(F['direction'],axis=1)).T
    bvecs[np.isnan(bvecs)] = 0
    bvals = F['bval']
    gtabExt = gradient_table(bvals = bvals,bvecs = bvecs)

    # Compute the mask where the sum is not zero
    dat = LesDict['data']
    mask = np.sum(dat[...,:69], axis=-1) != 0

    # Get the indices where mask is True
    indices = np.argwhere(mask)

    def optimize_chunk(pixels):
        results = []
        for i, j in pixels:
            samples = Network.sample((500,), x=DKIFeatures(gtabExt.bvecs[:],gtabExt.bvals[:],dat[i,j,:]),show_progress_bars=False)
            results.append((i, j, samples.mean(axis=0)))
        return results

    chunked_indices = [indices[i:i+ChunkSize] for i in range(0, len(indices), ChunkSize)]
    results = Parallel(n_jobs=8)(
        delayed(optimize_chunk)(chunk) for chunk in tqdm(chunked_indices,position=0,leave=True)
    )


    # Initialize NoiseEst with the appropriate shape
    ArrShape = mask.shape

    NoiseEst = np.zeros(list(ArrShape) + [22])
    for chunk in tqdm(results,position=0,leave=True):
        for i, j, x in chunk:
            NoiseEst[i, j] = x
    NoiseEst2 =  np.zeros_like(NoiseEst)
    for i in range(ArrShape[0]):
        for j in range(ArrShape[1]):    
            NoiseEst2[i,j] = np.hstack([mat_to_vals(clip_negative_eigenvalues(vals_to_mat(NoiseEst[i,j]))),NoiseEst[i,j,6:]])

    ArrShape = mask.shape

    FA_SBIFull  = np.zeros(ArrShape)
    MD_SBIFull  = np.zeros(ArrShape)        
    MK_SBIFull  = np.zeros(ArrShape)
    AK_SBIFull  = np.zeros(ArrShape)
    RK_SBIFull  = np.zeros(ArrShape)
    MKT_SBIFull = np.zeros(ArrShape)
    KFA_SBIFull = np.zeros(ArrShape)
    for i in tqdm(range(ArrShape[0]),position=0,leave=True):
        for j in range(ArrShape[1]):
            Metrics = DKIMetrics(NoiseEst2[i,j][:6],NoiseEst2[i,j][6:21])
            Eigs = np.linalg.eigh(vals_to_mat(NoiseEst2[i,j][:6]))[0]
            MD_SBIFull[i,j] = np.mean(Eigs)
            FA_SBIFull[i,j] = FracAni(Eigs,np.mean(Eigs))
            MK_SBIFull[i,j] = Metrics[0]
            AK_SBIFull[i,j] = Metrics[1]
            RK_SBIFull[i,j] = Metrics[2]
            MKT_SBIFull[i,j] = Metrics[3]
            KFA_SBIFull[i,j] = Metrics[4]
    KFA_SBIFull[np.isnan(KFA_SBIFull)] = 1

    RK_arr.append([RK_SBIFull[bin_les],RK_SBIFull[bin_con]])
    MK_arr.append([MK_SBIFull[bin_les],MK_SBIFull[bin_con]])
    AK_arr.append([AK_SBIFull[bin_les],AK_SBIFull[bin_con]])
    
    MD_arr.append([MD_SBIFull[bin_les],MD_SBIFull[bin_con]])
    FA_arr.append([FA_SBIFull[bin_les],FA_SBIFull[bin_con]])
    plt.boxplot(RK_SBIFull[bin_les],positions=[0],showfliers=False)
    plt.boxplot(RK_SBIFull[bin_con],positions=[1],showfliers=False)
    print(stats.ttest_ind(RK_SBIFull[bin_les],RK_SBIFull[bin_con]))
    plt.show()
    plt.boxplot(MK_SBIFull[bin_les],positions=[0],showfliers=False)
    plt.boxplot(MK_SBIFull[bin_con],positions=[1],showfliers=False)
    print(stats.ttest_ind(MK_SBIFull[bin_les],MK_SBIFull[bin_con]))
    plt.show()

In [ ]:
def add_sig_bar(ax, x1, x2, y, h, text):
    ax.plot([x1, x1, x2, x2], [y, y+h, y+h, y],
            lw=1, c='k')
    ax.text((x1+x2)/2, y, text,
            ha='center', va='center',fontsize=24)

In [ ]:
fig, ax = plt.subplots(figsize=(2.8,4.8))#, sharex=True)
les_RK = [R[0] for R in MK_arr]
con_RK = [R[1] for R in MK_arr]
# Plotting on ax1
ps = []
for i,(l1,l2) in enumerate(zip(les_RK,con_RK)):
    plt.sca(ax)
    y_data = l1
    g_pos = np.array([0.8+1*i])
    colors = ['red']
    colors2 = ['tomato']

    BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)
    
    y_data = l2
    g_pos = np.array([1 +1*i])
    colors = ['lightsalmon']
    colors2 = ['mistyrose']

    BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)
    ps.append(stats.mannwhitneyu(l1,l2).pvalue)
add_sig_bar(ax,0.8,1,1.8,0.0,'*')
add_sig_bar(ax,1.8,2,1.1,0.0,'*')
add_sig_bar(ax,3.8,4,1,0.0,'*')
ax.set_xticks([])

leg_patch2 = mpatches.Patch(color='red', label='Lesion')
leg_patch3 = mpatches.Patch(color='lightsalmon', label='Contralateral')

ax.legend(
    handles=[leg_patch2,leg_patch3],
    loc='upper right',         # base location  # fine-tune the legend's position
    frameon=False, ncols=1,
fontsize=24,columnspacing=0.3,handlelength=0.6,handletextpad=0.3,bbox_to_anchor=(1.2,1.))

In [ ]:
fig, ax = plt.subplots(figsize=(2.8,4.8))#, sharex=True)
les_RK = [R[0] for R in AK_arr]
con_RK = [R[1] for R in AK_arr]
# Plotting on ax1
ps = []
for i,(l1,l2) in enumerate(zip(les_RK,con_RK)):
    plt.sca(ax)
    y_data = l1
    g_pos = np.array([0.8+1*i])
    colors = ['red']
    colors2 = ['tomato']

    BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)
    
    y_data = l2
    g_pos = np.array([1 +1*i])
    colors = ['lightsalmon']
    colors2 = ['mistyrose']

    BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)
    ps.append(stats.mannwhitneyu(l1,l2).pvalue)
add_sig_bar(ax,0.8,1,1,0.0,'*')
add_sig_bar(ax,1.8,2,0.95,0.0,'*')
add_sig_bar(ax,3.8,4,0.87,0.0,'*')
ax.set_xticks([])


In [ ]:
fig, ax = plt.subplots(figsize=(2.8,4.8))#, sharex=True)
les_RK = [R[0] for R in RK_arr]
con_RK = [R[1] for R in RK_arr]
# Plotting on ax1
ps = []
for i,(l1,l2) in enumerate(zip(les_RK,con_RK)):
    plt.sca(ax)
    y_data = l1
    g_pos = np.array([0.8+1*i])
    colors = ['red']
    colors2 = ['tomato']

    BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)
    
    y_data = l2
    g_pos = np.array([1 +1*i])
    colors = ['lightsalmon']
    colors2 = ['mistyrose']

    BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)
    ps.append(stats.mannwhitneyu(l1,l2).pvalue)
add_sig_bar(ax,0.8,1,3.1,0.0,'*')
add_sig_bar(ax,1.8,2,1.3,0.0,'*')
add_sig_bar(ax,3.8,4,1.2,0.0,'*')
ax.set_xticks([])

In [ ]:
MK_arr_Min = []
AK_arr_Min = []
RK_arr_Min = []
FA_arr = []
MD_arr = []
for i,Name in tqdm(enumerate(['NMSS_11_1year','NMSS_15','NMSS_16','NMSS_18','NMSS_19'])):
    with open(MSDir+Name+'/Les_dict.pkl', 'rb') as f:
        LesDict = pickle.load(f)
    plt.imshow(LesDict['data'][...,0],cmap='gray')
    plt.imshow(LesDict['lesion'],cmap='autumn',alpha=0.5)
    plt.imshow(LesDict['contra'],cmap='winter',alpha=0.5)
    plt.show()
    plt.imshow(LesDict['data'][...,0],cmap='gray')
    plt.imshow(LesDict['lesion'],cmap='autumn',alpha=0.5)
    plt.imshow(np.fliplr(LesDict['contra']),cmap='winter',alpha=0.5)
    plt.show()
    plt.imshow(LesDict['lesion'],cmap='autumn')
    plt.imshow(LesDict['contra'],cmap='winter')
    plt.imshow(LesDict['contra']*LesDict['lesion'],cmap='gray')
    plt.show()
    bin_les = LesDict['lesion'] == LesDict['lesion']
    bin_con = LesDict['contra'] == LesDict['contra']
    overlap = bin_les*bin_con
    bin_les[overlap] = 0
    bin_con[overlap] = 0

    MatDir = MSDir+Name

    F = pmt.read_mat(MatDir+'/data_loaded.mat')
    bvecs = (F['direction'].T/np.linalg.norm(F['direction'],axis=1)).T
    bvecs[np.isnan(bvecs)] = 0
    bvals = F['bval']
    gtabExt = gradient_table(bvals = bvals,bvecs = bvecs)

    # Compute the mask where the sum is not zero
    dat = LesDict['data']
    mask = np.sum(dat[...,:69], axis=-1) != 0

    # Get the indices where mask is True
    indices = np.argwhere(mask)

    def optimize_chunk(pixels):
        results = []
        for i, j in pixels:
            samples = Network.sample((500,), x=DKIFeatures(gtabExt.bvecs[Indxs7],gtabExt.bvals[Indxs7],dat[i,j,Indxs7]),show_progress_bars=False)
            results.append((i, j, samples.mean(axis=0)))
        return results

    chunked_indices = [indices[i:i+ChunkSize] for i in range(0, len(indices), ChunkSize)]
    results = Parallel(n_jobs=8)(
        delayed(optimize_chunk)(chunk) for chunk in tqdm(chunked_indices,position=0,leave=True)
    )


    # Initialize NoiseEst with the appropriate shape
    ArrShape = mask.shape

    NoiseEst = np.zeros(list(ArrShape) + [22])
    for chunk in tqdm(results,position=0,leave=True):
        for i, j, x in chunk:
            NoiseEst[i, j] = x
    NoiseEst2 =  np.zeros_like(NoiseEst)
    for i in range(ArrShape[0]):
        for j in range(ArrShape[1]):    
            NoiseEst2[i,j] = np.hstack([mat_to_vals(clip_negative_eigenvalues(vals_to_mat(NoiseEst[i,j]))),NoiseEst[i,j,6:]])

    ArrShape = mask.shape

    FA_SBIMin  = np.zeros(ArrShape)
    MD_SBIMin  = np.zeros(ArrShape)        
    MK_SBIMin  = np.zeros(ArrShape)
    AK_SBIMin  = np.zeros(ArrShape)
    RK_SBIMin  = np.zeros(ArrShape)
    MKT_SBIMin = np.zeros(ArrShape)
    KFA_SBIMin = np.zeros(ArrShape)
    for i in tqdm(range(ArrShape[0]),position=0,leave=True):
        for j in range(ArrShape[1]):
            Metrics = DKIMetrics(NoiseEst2[i,j][:6],NoiseEst2[i,j][6:21])
            Eigs = np.linalg.eigh(vals_to_mat(NoiseEst2[i,j][:6]))[0]
            MD_SBIMin[i,j] = np.mean(Eigs)
            FA_SBIMin[i,j] = FracAni(Eigs,np.mean(Eigs))
            MK_SBIMin[i,j] = Metrics[0]
            AK_SBIMin[i,j] = Metrics[1]
            RK_SBIMin[i,j] = Metrics[2]
            MKT_SBIMin[i,j] = Metrics[3]
            KFA_SBIMin[i,j] = Metrics[4]
    KFA_SBIMin[np.isnan(KFA_SBIMin)] = 1

    RK_arr_Min.append([RK_SBIMin[bin_les],RK_SBIMin[bin_con]])
    MK_arr_Min.append([MK_SBIMin[bin_les],MK_SBIMin[bin_con]])
    AK_arr_Min.append([AK_SBIMin[bin_les],AK_SBIMin[bin_con]])
    
    MD_arr.append([MD_SBIMin[bin_les],MD_SBIMin[bin_con]])
    FA_arr.append([FA_SBIMin[bin_les],FA_SBIMin[bin_con]])
    plt.boxplot(RK_SBIMin[bin_les],positions=[0],showfliers=False)
    plt.boxplot(RK_SBIMin[bin_con],positions=[1],showfliers=False)
    print(stats.ttest_ind(RK_SBIMin[bin_les],RK_SBIMin[bin_con]))
    plt.show()
    plt.boxplot(MK_SBIMin[bin_les],positions=[0],showfliers=False)
    plt.boxplot(MK_SBIMin[bin_con],positions=[1],showfliers=False)
    print(stats.ttest_ind(MK_SBIMin[bin_les],MK_SBIMin[bin_con]))
    plt.show()

In [ ]:
def add_sig_bar(ax, x1, x2, y, h1,h2, text):
    ax.plot([x1, x1, x2, x2], [y-h1, y, y, y-h2],
            lw=1, c='k')
    ax.text((x1+x2)/2, y, text,
            ha='center', va='center',fontsize=24)

In [ ]:
fig, ax = plt.subplots(figsize=(2.8,4.8))#, sharex=True)
les_RK = [R[0] for R in MK_arr_Min]
con_RK = [R[1] for R in MK_arr_Min]
# Plotting on ax1
ps = []
for i,(l1,l2) in enumerate(zip(les_RK,con_RK)):
    plt.sca(ax)
    y_data = l1
    g_pos = np.array([0.8+1*i])
    colors = ['red']
    colors2 = ['tomato']

    BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)
    
    y_data = l2
    g_pos = np.array([1 +1*i])
    colors = ['lightsalmon']
    colors2 = ['mistyrose']

    BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)
    ps.append(stats.mannwhitneyu(l1,l2).pvalue)
add_sig_bar(ax,0.8,1,1.3,0.,'*')
add_sig_bar(ax,1.8,2,1.05,0.,'*')
add_sig_bar(ax,3.8,4.,1.07,0.,'*')
ax.set_xticks([])

leg_patch2 = mpatches.Patch(color='mediumturquoise', label='Lesion')
leg_patch3 = mpatches.Patch(color='sandybrown', label='Contralateral')

ax.set_yticks([0.3,0.6,0.9,1.2])

In [ ]:
fig, ax = plt.subplots(figsize=(2.8,4.8))#, sharex=True)
les_RK = [R[0] for R in AK_arr_Min]
con_RK = [R[1] for R in AK_arr_Min]
# Plotting on ax1
ps = []
for i,(l1,l2) in enumerate(zip(les_RK,con_RK)):
    plt.sca(ax)
    y_data = l1
    g_pos = np.array([0.8+1*i])
    colors = ['red']
    colors2 = ['tomato']

    BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)
    
    y_data = l2
    g_pos = np.array([1 +1*i])
    colors = ['lightsalmon']
    colors2 = ['mistyrose']

    BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)
    ps.append(stats.mannwhitneyu(l1,l2).pvalue)
add_sig_bar(ax,0.8,1,0.9,0.0,'*')
add_sig_bar(ax,1.8,2,0.95,0.0,'*')
add_sig_bar(ax,3.8,4,0.9,0.0,'*')
ax.set_xticks([])

In [ ]:
fig, ax = plt.subplots(figsize=(2.8,4.8))#, sharex=True)
les_RK = [R[0] for R in RK_arr_Min]
con_RK = [R[1] for R in RK_arr_Min]
# Plotting on ax1
ps = []
for i,(l1,l2) in enumerate(zip(les_RK,con_RK)):
    plt.sca(ax)
    y_data = l1
    g_pos = np.array([0.8+1*i])
    colors = ['red']
    colors2 = ['tomato']

    BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)
    
    y_data = l2
    g_pos = np.array([1 +1*i])
    colors = ['lightsalmon']
    colors2 = ['mistyrose']

    BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)
    ps.append(stats.mannwhitneyu(l1,l2).pvalue)
add_sig_bar(ax,0.8,1,1.9,0.0,'*')
add_sig_bar(ax,1.8,2,1.3,0.0,'*')
add_sig_bar(ax,3.8,4,1.3,0.0,'*')
ax.set_xticks([])
    

## j

In [ ]:
fig, ax = plt.subplots(figsize=(2.8,4.8))#, sharex=True)
les_RK = [R[0] for R in MK_arr]
con_RK = [R[1] for R in MK_arr]
y_data = np.array([(np.mean(lK)-np.mean(cK))/(np.mean(lK)+np.mean(cK)) for lK,cK in zip(les_RK,con_RK)])
g_pos = np.array([0.])
colors = ['black']
colors2 = ['dimgrey']

BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)

les_RK = [R[0] for R in MK_arr_Min]
con_RK = [R[1] for R in MK_arr_Min]
y_data = np.array([(np.mean(lK)-np.mean(cK))/(np.mean(lK)+np.mean(cK)) for lK,cK in zip(les_RK,con_RK)])
g_pos = np.array([0.2])
colors = ['gray']
colors2 = ['silver']

BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)

les_RK = [R[0] for R in AK_arr]
con_RK = [R[1] for R in AK_arr]
y_data = np.array([(np.mean(lK)-np.mean(cK))/(np.mean(lK)+np.mean(cK)) for lK,cK in zip(les_RK,con_RK)])
g_pos = np.array([1.])
colors = ['black']
colors2 = ['dimgrey']

BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)

les_RK = [R[0] for R in AK_arr_Min]
con_RK = [R[1] for R in AK_arr_Min]
y_data = np.array([(np.mean(lK)-np.mean(cK))/(np.mean(lK)+np.mean(cK)) for lK,cK in zip(les_RK,con_RK)])
g_pos = np.array([1.2])
colors = ['gray']
colors2 = ['silver']

BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)

les_RK = [R[0] for R in RK_arr]
con_RK = [R[1] for R in RK_arr]
y_data = np.array([(np.mean(lK)-np.mean(cK))/(np.mean(lK)+np.mean(cK)) for lK,cK in zip(les_RK,con_RK)])
g_pos = np.array([2.])
colors = ['black']
colors2 = ['dimgrey']

BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)

les_RK = [R[0] for R in RK_arr_Min]
con_RK = [R[1] for R in RK_arr_Min]
y_data = np.array([(np.mean(lK)-np.mean(cK))/(np.mean(lK)+np.mean(cK)) for lK,cK in zip(les_RK,con_RK)])
g_pos = np.array([2.2])
colors = ['gray']
colors2 = ['silver']

BoxPlots(y_data,g_pos,colors,colors2,ax,widths=0.2,scatter=True)

plt.xticks([0.1,1.1,2.1],['MK','RK','AK'])
plt.yticks([-0.1,0])

leg_patch2 = mpatches.Patch(color='black', label='Full')
leg_patch3 = mpatches.Patch(color='gray', label='Min')

ax.legend(
    handles=[leg_patch3,leg_patch2],
    loc='lower left',         # base location  # fine-tune the legend's position
    frameon=False, ncols=1,
fontsize=24,columnspacing=0.3,labelspacing=0.1,handlelength=0.6,handletextpad=0.3,bbox_to_anchor=(0.15,-0.08))
plt.axhline(0,ls='--',c='k',lw='2')